## Operando Mesh

In [ ]:
# -*- conding:utf-8 -*-
import sys
from pathlib import Path

import matplotlib as mpl
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
from matplotlib import gridspec, ticker
from matplotlib.colors import LinearSegmentedColormap
from tqdm.notebook import tqdm, trange


In [ ]:
# Ensure custom module Path is set before import
sys.path.append(r"D:\CHENG\OneDrive - UAB\ICMAB-Python\Figure")
from colors import tol_cmap, tol_cset  # type: ignore

# 画图的初始设置
plt.style.use(r"D:\CHENG\OneDrive - UAB\ICMAB-Python\Figure\liuchzzyy.mplstyle")
# print(plt.style.available)  # noqa: ERA001

# xarray setting
xr.set_options(
    cmap_sequential="viridis",
    cmap_divergent="viridis",
    display_width=150,
)  # viridis, gray

# 颜色设定
colors = tol_cset("vibrant")
if colors is not None:
    colors = list(colors)
else:
    # Fallback colors in case tol_cset returns None
    colors = ["#0077BB", "#33BBEE", "#009988", "#EE7733", "#CC3311", "#EE3377", "#BBBBBB"]
if r"sunset" not in plt.colormaps():
    cmap = tol_cmap("sunset")
    if isinstance(cmap, LinearSegmentedColormap):
        plt.colormaps.register(cmap)
if r"rainbow_PuRd" not in plt.colormaps():
    cmap = tol_cmap("rainbow_PuRd")
    if isinstance(cmap, LinearSegmentedColormap):
        plt.colormaps.register(cmap)  # 备用 plasma

# 输出的文件夹
path_out = Path(r"C:\Users\chengliu\Desktop\Figure")

# Set math font
mpl.rcParams["mathtext.fontset"] = "custom"
mpl.rcParams["mathtext.rm"] = "Arial"
mpl.rcParams["mathtext.it"] = "Arial:italic"
mpl.rcParams["mathtext.bf"] = "Arial:bold"
mpl.rcParams["mathtext.sf"] = "Arial"
mpl.rcParams["mathtext.tt"] = "Arial"
mpl.rcParams["mathtext.cal"] = "Arial"
mpl.rcParams["mathtext.default"] = "regular"

In [ ]:
import re


def create_folders(base_path: Path, folder_name: str) -> Path:
    folder_path = Path.joinpath(base_path, folder_name)
    if not folder_path.exists():
        folder_path.mkdir(parents=True, exist_ok=True)
    return folder_path


def data2map(df: pd.DataFrame, file_name: str, todisplay: bool = False, vmin: float = 2, vmax: float = 3.5):
    fig = plt.figure(file_name, dpi=100)
    ax = fig.add_subplot()
    p1 = ax.matshow(
        df.iloc[:, :],
        cmap="jet",
        interpolation="nearest",
        interpolation_stage="rgba",
        vmin=vmin,
        vmax=vmax + 1e-9,
        aspect=(0.067 / 0.032),
    )
    cbar = plt.colorbar(p1, fraction=0.05, pad=0.01)  # noqa: F841
    # cbar.ax.set_ylabel('Intensity', fontsize=9)  # noqa: ERA001
    ax.set_axis_off()

    if todisplay:
        display(df.head(5))
        plt.show()
    return fig


def parse_log_file(filepath):
    results = []
    current_sample = None
    start_time = None

    with open(filepath, "r", encoding="utf-8") as f:
        lines = f.readlines()

    for line in lines:
        timestamp = line[:23]
        msg = line[26:].strip()

        if "Scan sample" in msg:
            current_sample = re.sub(r"^INFO\s+", "", msg).replace("Scan sample ", "").strip()
            start_time = timestamp

        elif "Scan done:" in msg:
            match = re.search(r"scans_id:\s*(\d+)\s*-\s*(\d+)", msg)
            if match:
                scan_start = int(match.group(1))
                scan_end = int(match.group(2))
                results.append({
                    "Sample": current_sample,
                    "Start Time": start_time,
                    "End Time": timestamp,
                    "Scan ID Start": scan_start,
                    "Scan ID End": scan_end,
                })

    return results


def extract_experiment_metadata(folder_path, pattern=r"**\*.dat"):
    results = []

    for file_path in Path(folder_path).glob(pattern):
        with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
            lines = f.readlines()

        sample = file_path.stem.split("_")[0]
        element = file_path.stem.split("_")[-3]
        energy = file_path.stem.split("_")[-1]

        ScanID = []  # noqa: N806
        Time = []  # noqa: N806

        for line in lines:
            if line.startswith("#S "):
                # 提取 #S 行的第一个整数
                match = re.match(r"#S\s+(\d+)", line)
                if match:
                    ScanID.append(int(match.group(1)))
            elif line.startswith("#C") and "end" in line.lower():
                # 提取 'Acquisition ended at Wed Jul  5 16:50:07 2023' 中的时间
                match = re.search(r"\w{3} \w{3}\s+\d{1,2} \d{2}:\d{2}:\d{2} \d{4}", line)
                if match:
                    Time.append(match.group(0))

        for s, c in zip(ScanID, Time):
            results.append({"Sample": sample, "Element": element, 'Energy':energy, "ScanID": s, "Time": c})

    return pd.DataFrame(results)

### Echem, Mn4

#### 匹配谱线的时间和电化学的时间

In [ ]:
# 日志文件读取所有的时间和ID
log_folder = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XAS\Operando\αMnO2\MeshMapping\2023-CLAESS\Data"  # noqa: RUF001
)
results = []
for file in log_folder.glob("*.txt"):
    results.extend(parse_log_file(file))
exp_df = pd.DataFrame(results)
exp_df[["Scan ID Start", "Scan ID End"]] = exp_df[["Scan ID Start", "Scan ID End"]].apply(
    pd.to_numeric, errors="coerce"
    )  # noqa: E501
exp_df[["Start Time", 'End Time']] = exp_df[["Start Time", 'End Time']].apply(
    pd.to_datetime, format="mixed", errors="coerce"
    )
exp_df.to_csv(path_out.joinpath(r"log_metadata.csv"), index=False, header=True)  # noqa: ERA001

# 对应的实验的时间和ID
folder = r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XAS\Operando\αMnO2\MeshMapping\2023-CLAESS\Data\case2_1stDischarge"  # noqa: E501, RUF001
exp_df2 = extract_experiment_metadata(folder)
exp_df2[["Energy", "ScanID"]] = exp_df2[["Energy", "ScanID"]].apply(
    pd.to_numeric, errors="coerce"
)  # noqa: E501
exp_df2["Time"] = exp_df2["Time"].apply(pd.to_datetime, format="mixed", errors="coerce")
exp_df2.to_csv(path_out.joinpath(r"Time_Index_Spectrum.csv"), index=False, header=True)  # noqa: ERA001

In [ ]:
# 对应实验上的电化学时间
path_filelist = list(
    Path(
        r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XAS\Operando\αMnO2\MeshMapping\2023-CLAESS\Data\case2_1stDischarge\Echem"  # noqa: E501, RUF001
    ).glob(r"*up*.txt")
)
# 读取电化学数据
echem = []
for path_file in path_filelist:
    with open(path_file, "r", encoding="latin_1") as file:
        for line in file:
            if line.startswith("Nb header lines"):
                line_skip = int(line.split(":")[1].strip())
                break  # 发现后立即退出循环，提高效率

    df = pd.read_csv(
        path_file, sep="\t", comment="#", skiprows=line_skip - 1, encoding="latin_1", index_col=None, decimal="."
    ).dropna(axis=1, how="all")  # noqa: E501
    # # 转换数据格式
    df[["Ewe/V", "<I>/mA", "Capacity/mA.h"]] = df[["Ewe/V", "<I>/mA", "Capacity/mA.h"]].apply(
        pd.to_numeric, errors="coerce"
    )  # noqa: E501
    df["time/s"] = df["time/s"].apply(pd.to_datetime, format="mixed", errors="coerce")
    df["cycle number"] = df["cycle number"].astype(float).astype(np.int16)
    echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0, 1])]

In [ ]:
%matplotlib inline
plt.close("all")

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
labels = [
    [r"$\mathrm{1^{st}}$", None, None, None],
    [r"$\mathrm{2^{nd}}$", None, None, None],
    [r"$\mathrm{3^{rd}}$", None, None, None],
    [r"$\mathrm{4^{th}}$", None, None, None],
]
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx].reset_index(drop=True)
        # 找到电压最小值的索引
        idx_min = temp["Ewe/V"].idxmin()
        # 断开最小值前后的曲线
        ax.plot(
            temp.loc[:idx_min, "Capacity/mA.h"] * 1000 / 0.661,
            temp.loc[:idx_min, "Ewe/V"],
            ls="-",
            lw=1.0,
            c=colors[j],
            label=labels[j][i],
            zorder=0,
        )
        ax.plot(
            temp.loc[idx_min+5:temp.shape[0]-2, "Capacity/mA.h"] * 1000 / 0.661,
            temp.loc[idx_min+5:temp.shape[0]-2, "Ewe/V"],
            ls="-",
            lw=1.0,
            c=colors[j],
            label=None,
            zorder=0,
        )

# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(0, 300)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=50, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=25, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.8, 2.0)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.0))

ax.tick_params(axis="both", which='both', direction="out", labelsize=9)
ax.legend(loc="upper right", bbox_to_anchor=(1.0, 0.6), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)
ax.text(
    0.05,
    0.07,
    r"$\mathrm{0.5M \ ZnSO_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)

# 保存图片
plt.savefig(
    Path.joinpath(path_out, r"OperandoMesh_case2_Upcell_0_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"OperandoMesh_case2_Upcell_0_600.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor('white')
plt.show()

In [ ]:
data = pd.concat([echem[0]], ignore_index=True, axis=0)
data.drop(columns=["Unnamed: 3"], inplace=True, errors="ignore")  # noqa: ERA001

In [ ]:
# 谱线上的时间
path_file = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XAS\Operando\αMnO2\MeshMapping\2023-CLAESS\Results\V8\case2_1stDischarge"  # noqa: E501, RUF001
)
# 读取谱线的时间和ID
time_spectrum = pd.read_csv(
    Path.joinpath(path_file, r"Time_Index_Spectrum.csv"),
    sep=",",
    index_col=None,
    header=0,
    comment="#",
    date_format="%m/%d/%y %H:%M:%S.%f",
    parse_dates=[4],
)
time_spectrum["Time"] = pd.to_datetime(time_spectrum["Time"])
time_spectrum["ScanID"] = pd.to_numeric(time_spectrum["ScanID"])
# time_spectrum.info()  # noqa: ERA001

# 匹配谱线和电化学上的时间
echem_time = data["time/s"].values
target_times = time_spectrum.loc[
    (time_spectrum["Element"] == "Mn") &
    (time_spectrum["Sample"] == "UpCell"),
    "Time"
].values

index_spectrum_Mn = [  # noqa: N816
    np.abs(echem_time - t).argmin()
    for t in target_times
]

target_times = time_spectrum.loc[
    (time_spectrum["Element"] == "Zn") &
    (time_spectrum["Sample"] == "UpCell"),
    "Time"
].values
index_spectrum_Zn = [  # noqa: N816
    np.abs(echem_time - t).argmin()
    for t in target_times
]

# 提取对应的电位和电流数据，并按电位排序
Mn_info = time_spectrum.loc[
    (time_spectrum["Element"] == "Mn") &
    (time_spectrum["Sample"] == "UpCell"),
    ["Element", "Sample", "Energy"]
].reset_index(drop=True)

Mn_Voltage = data.loc[index_spectrum_Mn, ["Ewe/V", "<I>/mA"]].reset_index(drop=False)

index_voltage_Mn = pd.concat([Mn_Voltage, Mn_info], axis=1).reset_index(drop=False, inplace=False).sort_values(by="level_0")  # noqa: E501, N816

Zn_info = time_spectrum.loc[
    (time_spectrum["Element"] == "Zn") &
    (time_spectrum["Sample"] == "UpCell"),
    ["Element", "Sample", "Energy"]
].reset_index(drop=True)

Zn_Voltage = data.loc[index_spectrum_Zn, ["Ewe/V", "<I>/mA"]].reset_index(drop=False)

index_voltage_Zn = pd.concat([Zn_Voltage, Zn_info], axis=1).reset_index(drop=False, inplace=False).sort_values(by="level_0")  # noqa: E501, N816

In [ ]:
%matplotlib inline

# 画图
fig = plt.figure(figsize=(7.0, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.3)

ax.plot(data["time/s"], data["Ewe/V"], ls="-", lw=1.0, c=colors[0], label=r"Voltage", zorder=0)

# Mn
for i, j in enumerate(index_voltage_Mn['Energy'].unique()):
    selected_times = data["time/s"].loc[index_voltage_Mn.loc[index_voltage_Mn["Energy"] == j, "index"]]
    selected_voltages = data["Ewe/V"].loc[index_voltage_Mn["index"][index_voltage_Mn["Energy"] == j]]
    ax.scatter(selected_times, selected_voltages, c=colors[i], edgecolors="face", alpha=1.0, zorder=1)

# 添加索引文本标注
# 只标记一次，选取 r'6533' 的点
Index_special = [0, 5, 9, 15]
energy = index_voltage_Mn['Energy']
for i in [energy.unique()[0],]:

    row_index = np.array([
        np.abs(index_voltage_Mn[energy == i].loc[:, "level_0"] - val).argmin()
        for val in Index_special
    ])
    row_index = index_voltage_Mn[energy == i].iloc[row_index, 1]

    for j, idx in enumerate(row_index):
        ax.scatter(
            data["time/s"].iloc[idx], data["Ewe/V"].iloc[idx], c=colors[0], edgecolors="face", marker="o", zorder=2
        )
        ax.text(
            data["time/s"].iloc[idx],
            data["Ewe/V"].iloc[idx] + 0.03,
            str(Index_special[j]),
            fontsize=10,
            verticalalignment="bottom",
            horizontalalignment="right",
        )

# Zn
for i, j in enumerate(index_voltage_Zn['Energy'].unique()):
    selected_times = data["time/s"].loc[index_voltage_Zn.loc[index_voltage_Zn["Energy"] == j, "index"]]
    selected_voltages = data["Ewe/V"].loc[index_voltage_Zn["index"][index_voltage_Zn["Energy"] == j]]
    ax.scatter(selected_times, selected_voltages, c=colors[i], edgecolors="face", marker='*', alpha=1.0, zorder=1)


ax.set_ylabel(
    r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$",
    fontsize=11,
)
ax.set_ylim(0.85, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

# 确保时间刻度从数据最开始时间显示
ax.set_xlim(data["time/s"].min() - pd.Timedelta(minutes=20), data["time/s"].max() + pd.Timedelta(minutes=20))
ax.set_xlabel(r"Duration Time (hour)", fontsize=11, labelpad=5)
# ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))  # 设定日期格式为小时:分钟  # noqa: ERA001
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))  # 设定日期格式为小时:分钟
ax.xaxis.set_major_locator(mdates.HourLocator(byhour=range(0, 24, 1)))
ax.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=range(0, 60, 30)))
plt.xticks(rotation=60, horizontalalignment="right")  # 旋转刻度标签，防止重叠

ax.tick_params(
    axis="both", which="both", direction="out", labelsize=9, left=True, labelleft=True, top=False, labeltop=False
)
ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1), frameon=False, fontsize=11)

ax.text(
    0.17,
    0.95,
    r"$\text{1.322 mg cm}^{-2}$",
    transform=ax.transAxes,
    fontsize=12,
    color=colors[3],
    va="top",
    ha="right",
    fontfamily="Arial",
)

ax2 = ax.twinx()
ax2.set_position((0, 0, 1, 1))
ax2.set_box_aspect(0.3)

ax2.plot(data["time/s"], data["<I>/mA"], ls="--", lw=1.0, c=colors[3], label=r"Current", zorder=0)

ax2.set_ylabel(
    r"Current (mA)",
    fontsize=11,
)
ax2.set_ylim(-0.2, 0.2)
ax2.yaxis.set_major_locator(ticker.MultipleLocator(base=0.1, offset=0))
ax2.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.05, offset=0))
ax2.tick_params(axis="both", which="both", direction="out", labelsize=9, right=True, labelright=True)

ax2.legend(loc="upper left", bbox_to_anchor=(0.65, 1), frameon=False, fontsize=11)

plt.savefig(
    Path.joinpath(path_out, r"OperandoMesh_case2_Upcell_1_300_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"OperandoMesh_case2_Upcell_1_600_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

### Echem, Mn3

#### 匹配谱线的时间和电化学的时间

In [ ]:
# 日志文件读取所有的时间和ID
log_folder = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XAS\Operando\αMnO2\MeshMapping\2023-CLAESS\Data"  # noqa: RUF001
)
results = []
for file in log_folder.glob("*.txt"):
    results.extend(parse_log_file(file))
exp_df = pd.DataFrame(results)
exp_df[["Scan ID Start", "Scan ID End"]] = exp_df[["Scan ID Start", "Scan ID End"]].apply(
    pd.to_numeric, errors="coerce"
    )  # noqa: E501
exp_df[["Start Time", 'End Time']] = exp_df[["Start Time", 'End Time']].apply(
    pd.to_datetime, format="mixed", errors="coerce"
    )
exp_df.to_csv(path_out.joinpath(r"log_metadata.csv"), index=False, header=True)  # noqa: ERA001

# 对应的实验的时间和ID
folder = r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XAS\Operando\αMnO2\MeshMapping\2023-CLAESS\Data\case2_1stDischarge"  # noqa: E501, RUF001
exp_df2 = extract_experiment_metadata(folder)
exp_df2[["Energy", "ScanID"]] = exp_df2[["Energy", "ScanID"]].apply(
    pd.to_numeric, errors="coerce"
)  # noqa: E501
exp_df2["Time"] = exp_df2["Time"].apply(pd.to_datetime, format="mixed", errors="coerce")
exp_df2.to_csv(path_out.joinpath(r"Time_Index_Spectrum.csv"), index=False, header=True)  # noqa: ERA001

In [ ]:
# 对应实验上的电化学时间
path_filelist = list(
    Path(
        r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XAS\Operando\αMnO2\MeshMapping\2023-CLAESS\Data\case2_1stDischarge\Echem"  # noqa: E501, RUF001
    ).glob(r"*down*.txt")
)
# 读取电化学数据
echem = []
for path_file in path_filelist:
    with open(path_file, "r", encoding="latin_1") as file:
        for line in file:
            if line.startswith("Nb header lines"):
                line_skip = int(line.split(":")[1].strip())
                break  # 发现后立即退出循环，提高效率

    df = pd.read_csv(
        path_file, sep="\t", comment="#", skiprows=line_skip - 1, encoding="latin_1", index_col=None, decimal="."
    ).dropna(axis=1, how="all")  # noqa: E501
    # # 转换数据格式
    df[["Ewe/V", "<I>/mA", "Capacity/mA.h"]] = df[["Ewe/V", "<I>/mA", "Capacity/mA.h"]].apply(
        pd.to_numeric, errors="coerce"
    )  # noqa: E501
    df["time/s"] = df["time/s"].apply(pd.to_datetime, format="mixed", errors="coerce")
    df["cycle number"] = df["cycle number"].astype(float).astype(np.int16)
    echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0, 1])]

In [ ]:
%matplotlib inline
plt.close("all")

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
labels = [
    [r"$\mathrm{1^{st}}$", None, None, None],
    [r"$\mathrm{2^{nd}}$", None, None, None],
    [r"$\mathrm{3^{rd}}$", None, None, None],
    [r"$\mathrm{4^{th}}$", None, None, None],
]
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx].reset_index(drop=True)
        # 找到电压最小值的索引
        idx_min = temp["Ewe/V"].idxmin()
        # 断开最小值前后的曲线
        ax.plot(
            temp.loc[:idx_min, "Capacity/mA.h"] * 1000 / 0.653,
            temp.loc[:idx_min, "Ewe/V"],
            ls="-",
            lw=1.0,
            c=colors[j],
            label=labels[j][i],
            zorder=0,
        )
        ax.plot(
            temp.loc[idx_min+5:temp.shape[0]-2, "Capacity/mA.h"] * 1000 / 0.653,
            temp.loc[idx_min+5:temp.shape[0]-2, "Ewe/V"],
            ls="-",
            lw=1.0,
            c=colors[j],
            label=None,
            zorder=0,
        )

# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(0, 300)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=60, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=30, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.8, 2.0)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.0))

ax.tick_params(axis="both", which='both', direction="out", labelsize=9)
ax.legend(loc="upper right", bbox_to_anchor=(1.0, 0.6), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)
ax.text(
    0.05,
    0.07,
    r"$\mathrm{0.5M \ ZnSO_4 + 0.2M \ MnSO_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)

# 保存图片
plt.savefig(
    Path.joinpath(path_out, r"OperandoMesh_case2_Downcell_0_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"OperandoMesh_case2_Downcell_0_600.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor('white')
plt.show()

In [ ]:
data = pd.concat([echem[0]], ignore_index=True, axis=0)
data.drop(columns=["Unnamed: 3"], inplace=True, errors="ignore")  # noqa: ERA001

In [ ]:
# 谱线上的时间
path_file = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XAS\Operando\αMnO2\MeshMapping\2023-CLAESS\Results\V8\case2_1stDischarge"  # noqa: E501, RUF001
)
# 读取谱线的时间和ID
time_spectrum = pd.read_csv(
    Path.joinpath(path_file, r"Time_Index_Spectrum.csv"),
    sep=",",
    index_col=None,
    header=0,
    comment="#",
    date_format="%m/%d/%y %H:%M:%S.%f",
    parse_dates=[4],
)
time_spectrum["Time"] = pd.to_datetime(time_spectrum["Time"])
time_spectrum["ScanID"] = pd.to_numeric(time_spectrum["ScanID"])
# time_spectrum.info()  # noqa: ERA001

# 匹配谱线和电化学上的时间
echem_time = data["time/s"].values
target_times = time_spectrum.loc[
    (time_spectrum["Element"] == "Mn") &
    (time_spectrum["Sample"] == "DownCell"),
    "Time"
].values

index_spectrum_Mn = [  # noqa: N816
    np.abs(echem_time - t).argmin()
    for t in target_times
]

target_times = time_spectrum.loc[
    (time_spectrum["Element"] == "Zn") &
    (time_spectrum["Sample"] == "DownCell"),
    "Time"
].values
index_spectrum_Zn = [  # noqa: N816
    np.abs(echem_time - t).argmin()
    for t in target_times
]

# 提取对应的电位和电流数据，并按电位排序
Mn_info = time_spectrum.loc[
    (time_spectrum["Element"] == "Mn") &
    (time_spectrum["Sample"] == "DownCell"),
    ["Element", "Sample", "Energy"]
].reset_index(drop=True)

Mn_Voltage = data.loc[index_spectrum_Mn, ["Ewe/V", "<I>/mA"]].reset_index(drop=False)

index_voltage_Mn = pd.concat([Mn_Voltage, Mn_info], axis=1).reset_index(drop=False, inplace=False).sort_values(by="level_0")  # noqa: E501, N816

Zn_info = time_spectrum.loc[
    (time_spectrum["Element"] == "Zn") &
    (time_spectrum["Sample"] == "DownCell"),
    ["Element", "Sample", "Energy"]
].reset_index(drop=True)

Zn_Voltage = data.loc[index_spectrum_Zn, ["Ewe/V", "<I>/mA"]].reset_index(drop=False)

index_voltage_Zn = pd.concat([Zn_Voltage, Zn_info], axis=1).reset_index(drop=False, inplace=False).sort_values(by="level_0")  # noqa: E501, N816

In [ ]:
%matplotlib inline

# 画图
fig = plt.figure(figsize=(7.0, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.3)

ax.plot(data["time/s"], data["Ewe/V"], ls="-", lw=1.0, c=colors[0], label=r"Voltage", zorder=0)

# Mn
for i, j in enumerate(index_voltage_Mn['Energy'].unique()):
    selected_times = data["time/s"].loc[index_voltage_Mn.loc[index_voltage_Mn["Energy"] == j, "index"]]
    selected_voltages = data["Ewe/V"].loc[index_voltage_Mn["index"][index_voltage_Mn["Energy"] == j]]
    ax.scatter(selected_times, selected_voltages, c=colors[i], edgecolors="face", alpha=1.0, zorder=1)

# 添加索引文本标注
# 只标记一次，选取 r'6533' 的点
Index_special = [0, 5, 9, 15]
energy = index_voltage_Mn['Energy']
for i in [energy.unique()[0],]:
    row_index = np.array([
        np.abs(index_voltage_Mn[energy == i].loc[:, "level_0"] - val).argmin()
        for val in Index_special
    ])
    row_index = index_voltage_Mn[energy == i].iloc[row_index, 1]

    for j, idx in enumerate(row_index):
        ax.scatter(
            data["time/s"].iloc[idx], data["Ewe/V"].iloc[idx], c=colors[0], edgecolors="face", marker="o", zorder=2
        )
        ax.text(
            data["time/s"].iloc[idx],
            data["Ewe/V"].iloc[idx] + 0.03,
            str(Index_special[j]),
            fontsize=10,
            verticalalignment="bottom",
            horizontalalignment="right",
        )

# Zn
for i, j in enumerate(index_voltage_Zn['Energy'].unique()):
    selected_times = data["time/s"].loc[index_voltage_Zn.loc[index_voltage_Zn["Energy"] == j, "index"]]
    selected_voltages = data["Ewe/V"].loc[index_voltage_Zn["index"][index_voltage_Zn["Energy"] == j]]
    ax.scatter(selected_times, selected_voltages, c=colors[i], edgecolors="face", marker='*', alpha=1.0, zorder=1)


ax.set_ylabel(
    r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$",
    fontsize=11,
)
ax.set_ylim(0.85, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

# 确保时间刻度从数据最开始时间显示
ax.set_xlim(data["time/s"].min() - pd.Timedelta(minutes=20), data["time/s"].max() + pd.Timedelta(minutes=20))
ax.set_xlabel(r"Duration Time (hour)", fontsize=11, labelpad=5)
# ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))  # 设定日期格式为小时:分钟  # noqa: ERA001
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))  # 设定日期格式为小时:分钟
ax.xaxis.set_major_locator(mdates.HourLocator(byhour=range(0, 24, 1)))
ax.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=range(0, 60, 30)))
plt.xticks(rotation=60, horizontalalignment="right")  # 旋转刻度标签，防止重叠

ax.tick_params(
    axis="both", which="both", direction="out", labelsize=9, left=True, labelleft=True, top=False, labeltop=False
)
ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1), frameon=False, fontsize=11)

ax.text(
    0.17,
    0.95,
    r"$\text{1.306 mg cm}^{-2}$",
    transform=ax.transAxes,
    fontsize=12,
    color=colors[3],
    va="top",
    ha="right",
    fontfamily="Arial",
)

ax2 = ax.twinx()
ax2.set_position((0, 0, 1, 1))
ax2.set_box_aspect(0.3)

ax2.plot(data["time/s"], data["<I>/mA"], ls="--", lw=1.0, c=colors[3], label=r"Current", zorder=0)

ax2.set_ylabel(
    r"Current (mA)",
    fontsize=11,
)
ax2.set_ylim(-0.2, 0.2)
ax2.yaxis.set_major_locator(ticker.MultipleLocator(base=0.1, offset=0))
ax2.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.05, offset=0))
ax2.tick_params(axis="both", which="both", direction="out", labelsize=9, right=True, labelright=True)

ax2.legend(loc="upper left", bbox_to_anchor=(0.65, 1), frameon=False, fontsize=11)

plt.savefig(
    Path.joinpath(path_out, r"OperandoMesh_case2_Downcell_1_300_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"OperandoMesh_case2_Downcell_1_600_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

### Echem, Mn2

#### 匹配谱线的时间和电化学的时间

In [ ]:
# 日志文件读取所有的时间和ID
log_folder = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XAS\Operando\αMnO2\MeshMapping\2023-CLAESS\Data"  # noqa: RUF001
)
results = []
for file in log_folder.glob("*.txt"):
    results.extend(parse_log_file(file))
exp_df = pd.DataFrame(results)
exp_df[["Scan ID Start", "Scan ID End"]] = exp_df[["Scan ID Start", "Scan ID End"]].apply(
    pd.to_numeric, errors="coerce"
    )  # noqa: E501
exp_df[["Start Time", 'End Time']] = exp_df[["Start Time", 'End Time']].apply(
    pd.to_datetime, format="mixed", errors="coerce"
    )
exp_df.to_csv(path_out.joinpath(r"log_metadata.csv"), index=False, header=True)  # noqa: ERA001

# 对应的实验的时间和ID
folder = r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XAS\Operando\αMnO2\MeshMapping\2023-CLAESS\Data\case1_1stCharge_1stDischarge"  # noqa: E501, RUF001
exp_df2 = extract_experiment_metadata(folder)
exp_df2[["Energy", "ScanID"]] = exp_df2[["Energy", "ScanID"]].apply(
    pd.to_numeric, errors="coerce"
)  # noqa: E501
exp_df2["Time"] = exp_df2["Time"].apply(pd.to_datetime, format="mixed", errors="coerce")
exp_df2.to_csv(path_out.joinpath(r"Time_Index_Spectrum.csv"), index=False, header=True)  # noqa: ERA001

In [ ]:
# 对应实验上的电化学时间
path_filelist = list(
    Path(
        r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XAS\Operando\αMnO2\MeshMapping\2023-CLAESS\Data\case1_1stCharge_1stDischarge\Echem"  # noqa: E501, RUF001
    ).glob(r"*up*.txt")
)
# 读取电化学数据
echem = []
for path_file in path_filelist:
    with open(path_file, "r", encoding="latin_1") as file:
        for line in file:
            if line.startswith("Nb header lines"):
                line_skip = int(line.split(":")[1].strip())
                break  # 发现后立即退出循环，提高效率

    df = pd.read_csv(
        path_file, sep="\t", comment="#", skiprows=line_skip - 1, encoding="latin_1", index_col=None, decimal="."
    ).dropna(axis=1, how="all")  # noqa: E501
    # # 转换数据格式
    df[["Ewe/V", "<I>/mA", "Capacity/mA.h"]] = df[["Ewe/V", "<I>/mA", "Capacity/mA.h"]].apply(
        pd.to_numeric, errors="coerce"
    )  # noqa: E501
    df["time/s"] = df["time/s"].apply(pd.to_datetime, format="mixed", errors="coerce")
    df["cycle number"] = df["cycle number"].astype(float).astype(np.int16)
    echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0, 1])]

In [ ]:
%matplotlib inline
plt.close("all")

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
labels = [
    [r"$\mathrm{1^{st}}$", None, None, None],
    [r"$\mathrm{2^{nd}}$", None, None, None],
    [r"$\mathrm{3^{rd}}$", None, None, None],
    [r"$\mathrm{4^{th}}$", None, None, None],
]
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx].reset_index(drop=True)
        # 找到电压最小值的索引
        idx_min = temp["Ewe/V"].idxmin()
        # 断开最小值前后的曲线
        ax.plot(
            temp.loc[:idx_min, "Capacity/mA.h"] * 1000 / 0.582,
            temp.loc[:idx_min, "Ewe/V"],
            ls="-",
            lw=1.0,
            c=colors[j],
            label=labels[j][i],
            zorder=0,
        )
        ax.plot(
            temp.loc[idx_min+5:temp.shape[0]-2, "Capacity/mA.h"] * 1000 / 0.582,
            temp.loc[idx_min+5:temp.shape[0]-2, "Ewe/V"],
            ls="-",
            lw=1.0,
            c=colors[j],
            label=None,
            zorder=0,
        )

# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(0, 300)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=50, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=25, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.8, 2.0)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.0))

ax.tick_params(axis="both", which='both', direction="out", labelsize=9)
ax.legend(loc="upper right", bbox_to_anchor=(1.0, 0.6), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)
ax.text(
    0.05,
    0.07,
    r"$\mathrm{0.5M \ ZnSO_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)

# 保存图片
plt.savefig(
    Path.joinpath(path_out, r"OperandoMesh_case1_Upcell_0_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"OperandoMesh_case1_Upcell_0_600.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor('white')
plt.show()

In [ ]:
data = pd.concat([echem[1], echem[0]], ignore_index=True, axis=0)
data.drop(columns=["Unnamed: 3"], inplace=True, errors="ignore")  # noqa: ERA001

In [ ]:
# 谱线上的时间
path_file = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XAS\Operando\αMnO2\MeshMapping\2023-CLAESS\Results\V8\case1_1stCharge_1stDischarge"  # noqa: E501, RUF001
)
# 读取谱线的时间和ID
time_spectrum = pd.read_csv(
    Path.joinpath(path_file, r"Time_Index_Spectrum.csv"),
    sep=",",
    index_col=None,
    header=0,
    comment="#",
    date_format="%m/%d/%y %H:%M:%S.%f",
    parse_dates=[4],
)
time_spectrum["Time"] = pd.to_datetime(time_spectrum["Time"])
time_spectrum["ScanID"] = pd.to_numeric(time_spectrum["ScanID"])
# time_spectrum.info()  # noqa: ERA001

# 匹配谱线和电化学上的时间
echem_time = data["time/s"].values
target_times = time_spectrum.loc[
    (time_spectrum["Element"] == "Mn") &
    (time_spectrum["Sample"] == "DownCell"),
    "Time"
].values

index_spectrum_Mn = [  # noqa: N816
    np.abs(echem_time - t).argmin()
    for t in target_times
]

target_times = time_spectrum.loc[
    (time_spectrum["Element"] == "Zn") &
    (time_spectrum["Sample"] == "DownCell"),
    "Time"
].values
index_spectrum_Zn = [  # noqa: N816
    np.abs(echem_time - t).argmin()
    for t in target_times
]

# 提取对应的电位和电流数据，并按电位排序
Mn_info = time_spectrum.loc[
    (time_spectrum["Element"] == "Mn") &
    (time_spectrum["Sample"] == "DownCell"),
    ["Element", "Sample", "Energy"]
].reset_index(drop=True)

Mn_Voltage = data.loc[index_spectrum_Mn, ["Ewe/V", "<I>/mA"]].reset_index(drop=False)

index_voltage_Mn = pd.concat([Mn_Voltage, Mn_info], axis=1).reset_index(drop=False, inplace=False).sort_values(by="level_0")  # noqa: E501, N816

Zn_info = time_spectrum.loc[
    (time_spectrum["Element"] == "Zn") &
    (time_spectrum["Sample"] == "DownCell"),
    ["Element", "Sample", "Energy"]
].reset_index(drop=True)

Zn_Voltage = data.loc[index_spectrum_Zn, ["Ewe/V", "<I>/mA"]].reset_index(drop=False)

index_voltage_Zn = pd.concat([Zn_Voltage, Zn_info], axis=1).reset_index(drop=False, inplace=False).sort_values(by="level_0")  # noqa: E501, N816

In [ ]:
%matplotlib inline

# 画图
fig = plt.figure(figsize=(7.0, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.3)

ax.plot(data["time/s"], data["Ewe/V"], ls="-", lw=1.0, c=colors[0], label=r"Voltage", zorder=0)

# Mn
for i, j in enumerate(index_voltage_Mn['Energy'].unique()):
    selected_times = data["time/s"].loc[index_voltage_Mn.loc[index_voltage_Mn["Energy"] == j, "index"]]
    selected_voltages = data["Ewe/V"].loc[index_voltage_Mn["index"][index_voltage_Mn["Energy"] == j]]
    ax.scatter(selected_times, selected_voltages, c=colors[i], edgecolors="face", alpha=1.0, zorder=1)

# 添加索引文本标注
# 只标记一次，选取 r'6533' 的点
Index_special = [0, 5, 8, 12, 17, 22, 27, 32, 40]
energy = index_voltage_Mn['Energy']
for i in [energy.unique()[0],]:
    row_index = np.array([
        np.abs(index_voltage_Mn[energy == i].loc[:, "level_0"] - val).argmin()
        for val in Index_special
    ])
    row_index = index_voltage_Mn[energy == i].iloc[row_index, 1]

    for j, idx in enumerate(row_index):
        ax.scatter(
            data["time/s"].iloc[idx], data["Ewe/V"].iloc[idx], c=colors[0], edgecolors="face", marker="o", zorder=2
        )
        ax.text(
            data["time/s"].iloc[idx],
            data["Ewe/V"].iloc[idx] + 0.03,
            str(Index_special[j]),
            fontsize=10,
            verticalalignment="bottom",
            horizontalalignment="right",
        )

# Zn
for i, j in enumerate(index_voltage_Zn['Energy'].unique()):
    selected_times = data["time/s"].loc[index_voltage_Zn.loc[index_voltage_Zn["Energy"] == j, "index"]]
    selected_voltages = data["Ewe/V"].loc[index_voltage_Zn["index"][index_voltage_Zn["Energy"] == j]]
    ax.scatter(selected_times, selected_voltages, c=colors[i], edgecolors="face", marker='*', alpha=1.0, zorder=1)


ax.set_ylabel(
    r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$",
    fontsize=11,
)
ax.set_ylim(0.85, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

# 确保时间刻度从数据最开始时间显示
ax.set_xlim(data["time/s"].min() - pd.Timedelta(minutes=20), data["time/s"].max() + pd.Timedelta(minutes=20))
ax.set_xlabel(r"Duration Time (hour)", fontsize=11, labelpad=5)
# ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))  # 设定日期格式为小时:分钟  # noqa: ERA001
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))  # 设定日期格式为小时:分钟
ax.xaxis.set_major_locator(mdates.HourLocator(byhour=range(0, 24, 1)))
ax.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=range(0, 60, 30)))
plt.xticks(rotation=60, horizontalalignment="right")  # 旋转刻度标签，防止重叠

ax.tick_params(
    axis="both", which="both", direction="out", labelsize=9, left=True, labelleft=True, top=False, labeltop=False
)
ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1), frameon=False, fontsize=11)

ax.text(
    0.17,
    0.95,
    r"$\text{1.164 mg cm}^{-2}$",
    transform=ax.transAxes,
    fontsize=12,
    color=colors[3],
    va="top",
    ha="right",
    fontfamily="Arial",
)

ax2 = ax.twinx()
ax2.set_position((0, 0, 1, 1))
ax2.set_box_aspect(0.3)

ax2.plot(data["time/s"], data["<I>/mA"], ls="--", lw=1.0, c=colors[3], label=r"Current", zorder=0)

ax2.set_ylabel(
    r"Current (mA)",
    fontsize=11,
)
ax2.set_ylim(-0.2, 0.2)
ax2.yaxis.set_major_locator(ticker.MultipleLocator(base=0.1, offset=0))
ax2.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.05, offset=0))
ax2.tick_params(axis="both", which="both", direction="out", labelsize=9, right=True, labelright=True)

ax2.legend(loc="upper left", bbox_to_anchor=(0.65, 1), frameon=False, fontsize=11)

plt.savefig(
    Path.joinpath(path_out, r"OperandoMesh_case1_Upcell_1_300_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"OperandoMesh_case1_Upcell_1_600_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

### Echem, Mn1

#### 匹配谱线的时间和电化学的时间

In [ ]:
# 日志文件读取所有的时间和ID
log_folder = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XAS\Operando\αMnO2\MeshMapping\2023-CLAESS\Data"  # noqa: RUF001
)
results = []
for file in log_folder.glob("*.txt"):
    results.extend(parse_log_file(file))
exp_df = pd.DataFrame(results)
exp_df[["Scan ID Start", "Scan ID End"]] = exp_df[["Scan ID Start", "Scan ID End"]].apply(
    pd.to_numeric, errors="coerce"
    )  # noqa: E501
exp_df[["Start Time", 'End Time']] = exp_df[["Start Time", 'End Time']].apply(
    pd.to_datetime, format="mixed", errors="coerce"
    )
exp_df.to_csv(path_out.joinpath(r"log_metadata.csv"), index=False, header=True)  # noqa: ERA001

# 对应的实验的时间和ID
folder = r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XAS\Operando\αMnO2\MeshMapping\2023-CLAESS\Data\case1_1stCharge_1stDischarge"  # noqa: E501, RUF001
exp_df2 = extract_experiment_metadata(folder)
exp_df2[["Energy", "ScanID"]] = exp_df2[["Energy", "ScanID"]].apply(
    pd.to_numeric, errors="coerce"
)  # noqa: E501
exp_df2["Time"] = exp_df2["Time"].apply(pd.to_datetime, format="mixed", errors="coerce")
exp_df2.to_csv(path_out.joinpath(r"Time_Index_Spectrum.csv"), index=False, header=True)  # noqa: ERA001

In [ ]:
# 对应实验上的电化学时间
path_filelist = list(
    Path(
        r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XAS\Operando\αMnO2\MeshMapping\2023-CLAESS\Data\case1_1stCharge_1stDischarge\Echem"  # noqa: E501, RUF001
    ).glob(r"*down*.txt")
)
# 读取电化学数据
echem = []
for path_file in path_filelist:
    with open(path_file, "r", encoding="latin_1") as file:
        for line in file:
            if line.startswith("Nb header lines"):
                line_skip = int(line.split(":")[1].strip())
                break  # 发现后立即退出循环，提高效率

    df = pd.read_csv(
        path_file, sep="\t", comment="#", skiprows=line_skip - 1, encoding="latin_1", index_col=None, decimal="."
    ).dropna(axis=1, how="all")  # noqa: E501
    # # 转换数据格式
    df[["Ewe/V", "<I>/mA", "Capacity/mA.h"]] = df[["Ewe/V", "<I>/mA", "Capacity/mA.h"]].apply(
        pd.to_numeric, errors="coerce"
    )  # noqa: E501
    df["time/s"] = df["time/s"].apply(pd.to_datetime, format="mixed", errors="coerce")
    df["cycle number"] = df["cycle number"].astype(float).astype(np.int16)
    echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0, 1])]

In [ ]:
%matplotlib inline
plt.close("all")

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
labels = [
    [r"$\mathrm{1^{st}}$", None, None, None],
    [r"$\mathrm{2^{nd}}$", None, None, None],
    [r"$\mathrm{3^{rd}}$", None, None, None],
    [r"$\mathrm{4^{th}}$", None, None, None],
]
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx].reset_index(drop=True)
        # 找到电压最小值的索引
        idx_min = temp["Ewe/V"].idxmin()
        # 断开最小值前后的曲线
        ax.plot(
            temp.loc[:idx_min, "Capacity/mA.h"] * 1000 / 0.821,
            temp.loc[:idx_min, "Ewe/V"],
            ls="-",
            lw=1.0,
            c=colors[j],
            label=labels[j][i],
            zorder=0,
        )
        ax.plot(
            temp.loc[idx_min+5:temp.shape[0]-2, "Capacity/mA.h"] * 1000 / 0.821,
            temp.loc[idx_min+5:temp.shape[0]-2, "Ewe/V"],
            ls="-",
            lw=1.0,
            c=colors[j],
            label=None,
            zorder=0,
        )

# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(0, 400)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=80, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=40, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.8, 2.0)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.0))

ax.tick_params(axis="both", which='both', direction="out", labelsize=9)
ax.legend(loc="upper right", bbox_to_anchor=(1.0, 0.6), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)
ax.text(
    0.05,
    0.07,
    r"$\mathrm{0.5M \ ZnSO_4 + 0.2M \ MnSO_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)

# 保存图片
plt.savefig(
    Path.joinpath(path_out, r"OperandoMesh_case1_Downcell_0_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"OperandoMesh_case1_Downcell_0_600.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor('white')
plt.show()

In [ ]:
data = pd.concat([echem[1], echem[0]], ignore_index=True, axis=0)
data.drop(columns=["Unnamed: 3"], inplace=True, errors="ignore")  # noqa: ERA001

In [ ]:
# 谱线上的时间
path_file = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XAS\Operando\αMnO2\MeshMapping\2023-CLAESS\Results\V8\case1_1stCharge_1stDischarge"  # noqa: E501, RUF001
)
# 读取谱线的时间和ID
time_spectrum = pd.read_csv(
    Path.joinpath(path_file, r"Time_Index_Spectrum.csv"),
    sep=",",
    index_col=None,
    header=0,
    comment="#",
    date_format="%m/%d/%y %H:%M:%S.%f",
    parse_dates=[4],
)
time_spectrum["Time"] = pd.to_datetime(time_spectrum["Time"])
time_spectrum["ScanID"] = pd.to_numeric(time_spectrum["ScanID"])
# time_spectrum.info()  # noqa: ERA001

# 匹配谱线和电化学上的时间
echem_time = data["time/s"].values
target_times = time_spectrum.loc[
    (time_spectrum["Element"] == "Mn") &
    (time_spectrum["Sample"] == "DownCell"),
    "Time"
].values

index_spectrum_Mn = [  # noqa: N816
    np.abs(echem_time - t).argmin()
    for t in target_times
]

target_times = time_spectrum.loc[
    (time_spectrum["Element"] == "Zn") &
    (time_spectrum["Sample"] == "DownCell"),
    "Time"
].values
index_spectrum_Zn = [  # noqa: N816
    np.abs(echem_time - t).argmin()
    for t in target_times
]

# 提取对应的电位和电流数据，并按电位排序
Mn_info = time_spectrum.loc[
    (time_spectrum["Element"] == "Mn") &
    (time_spectrum["Sample"] == "DownCell"),
    ["Element", "Sample", "Energy"]
].reset_index(drop=True)

Mn_Voltage = data.loc[index_spectrum_Mn, ["Ewe/V", "<I>/mA"]].reset_index(drop=False)

index_voltage_Mn = pd.concat([Mn_Voltage, Mn_info], axis=1).reset_index(drop=False, inplace=False).sort_values(by="level_0")  # noqa: E501, N816

Zn_info = time_spectrum.loc[
    (time_spectrum["Element"] == "Zn") &
    (time_spectrum["Sample"] == "DownCell"),
    ["Element", "Sample", "Energy"]
].reset_index(drop=True)

Zn_Voltage = data.loc[index_spectrum_Zn, ["Ewe/V", "<I>/mA"]].reset_index(drop=False)

index_voltage_Zn = pd.concat([Zn_Voltage, Zn_info], axis=1).reset_index(drop=False, inplace=False).sort_values(by="level_0")  # noqa: E501, N816

In [ ]:
%matplotlib inline

# 画图
fig = plt.figure(figsize=(7.0, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.3)

ax.plot(data["time/s"], data["Ewe/V"], ls="-", lw=1.0, c=colors[0], label=r"Voltage", zorder=0)

# Mn
for i, j in enumerate(index_voltage_Mn['Energy'].unique()):
    selected_times = data["time/s"].loc[index_voltage_Mn.loc[index_voltage_Mn["Energy"] == j, "index"]]
    selected_voltages = data["Ewe/V"].loc[index_voltage_Mn["index"][index_voltage_Mn["Energy"] == j]]
    ax.scatter(selected_times, selected_voltages, c=colors[i], edgecolors="face", alpha=1.0, zorder=1)

# 添加索引文本标注
# 只标记一次，选取 r'6533' 的点
Index_special = [0, 5, 9, 15, 19, 27, 34, 41]
energy = index_voltage_Mn['Energy']
for i in [energy.unique()[0],]:
    row_index = np.array([
        np.abs(index_voltage_Mn[energy == i].loc[:, "level_0"] - val).argmin()
        for val in Index_special
    ])
    row_index = index_voltage_Mn[energy == i].iloc[row_index, 1]

    for j, idx in enumerate(row_index):
        ax.scatter(
            data["time/s"].iloc[idx], data["Ewe/V"].iloc[idx], c=colors[0], edgecolors="face", marker="o", zorder=2
        )
        ax.text(
            data["time/s"].iloc[idx],
            data["Ewe/V"].iloc[idx] + 0.03,
            str(Index_special[j]),
            fontsize=10,
            verticalalignment="bottom",
            horizontalalignment="right",
        )

# Zn
for i, j in enumerate(index_voltage_Zn['Energy'].unique()):
    selected_times = data["time/s"].loc[index_voltage_Zn.loc[index_voltage_Zn["Energy"] == j, "index"]]
    selected_voltages = data["Ewe/V"].loc[index_voltage_Zn["index"][index_voltage_Zn["Energy"] == j]]
    ax.scatter(selected_times, selected_voltages, c=colors[i], edgecolors="face", marker='*', alpha=1.0, zorder=1)


ax.set_ylabel(
    r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$",
    fontsize=11,
)
ax.set_ylim(0.85, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

# 确保时间刻度从数据最开始时间显示
ax.set_xlim(data["time/s"].min() - pd.Timedelta(minutes=20), data["time/s"].max() + pd.Timedelta(minutes=20))
ax.set_xlabel(r"Duration Time (hour)", fontsize=11, labelpad=5)
# ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))  # 设定日期格式为小时:分钟  # noqa: ERA001
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))  # 设定日期格式为小时:分钟
ax.xaxis.set_major_locator(mdates.HourLocator(byhour=range(0, 24, 1)))
ax.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=range(0, 60, 30)))
plt.xticks(rotation=60, horizontalalignment="right")  # 旋转刻度标签，防止重叠

ax.tick_params(
    axis="both", which="both", direction="out", labelsize=9, left=True, labelleft=True, top=False, labeltop=False
)
ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1), frameon=False, fontsize=11)

ax.text(
    0.17,
    0.95,
    r"$\text{1.642 mg cm}^{-2}$",
    transform=ax.transAxes,
    fontsize=12,
    color=colors[3],
    va="top",
    ha="right",
    fontfamily="Arial",
)

ax2 = ax.twinx()
ax2.set_position((0, 0, 1, 1))
ax2.set_box_aspect(0.3)

ax2.plot(data["time/s"], data["<I>/mA"], ls="--", lw=1.0, c=colors[3], label=r"Current", zorder=0)

ax2.set_ylabel(
    r"Current (mA)",
    fontsize=11,
)
ax2.set_ylim(-0.2, 0.2)
ax2.yaxis.set_major_locator(ticker.MultipleLocator(base=0.1, offset=0))
ax2.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.05, offset=0))
ax2.tick_params(axis="both", which="both", direction="out", labelsize=9, right=True, labelright=True)

ax2.legend(loc="upper left", bbox_to_anchor=(0.65, 1), frameon=False, fontsize=11)

plt.savefig(
    Path.joinpath(path_out, r"OperandoMesh_case1_Downcell_1_300_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"OperandoMesh_case1_Downcell_1_600_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

### Mn4, a+b=1

#### 读取数据

In [ ]:
# to read all master files
path_master_file = list(
    Path(
        r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XAS\Operando\αMnO2\MeshMapping\2023-CLAESS\Data\case2_1stDischarge\UpCell_MnFree_2\Mn"  # noqa: E501, RUF001
    ).glob("*.dat")
)
header = "Pt_No tripod_z tripod_x energyc a_i0i1_timer a_i0_1 a_i0_2 a_i1_1 a_i1_2 n_timer n_i0_1 n_i0_2 n_i1_1 n_i1_2 n_i2_1 n_i2_2 n_icr n_tcr n_sca1 n_sca2 n_sca3 n_sca4 sr_i_1 dt"  # noqa: E501
headers = header.split(" ")

In [ ]:
# 处理 mesh 数据
dfout = []
scan_folder = create_folders(path_out, "0_PureAbsorption")
# read data files and clean it
for master_file_name in tqdm(path_master_file):
    name_folder = master_file_name.parts[-1][:-4]

    dfin = pd.read_csv(master_file_name, index_col=None, header=None, names=headers, sep=r"\s+", comment="#")

    NaNindex = dfin.isnull().any(axis=1).to_numpy().nonzero()[0].tolist()
    dfin.drop(NaNindex, axis=0, inplace=True)

    # 计算 Absorption
    i0 = dfin.loc[:, ["a_i0_1", "a_i0_2"]].sum(axis=1).values
    i1 = dfin.loc[:, ["a_i1_1", "a_i1_2"]].sum(axis=1).values
    dfin["absorption"] = np.log(i0 / i1)  # type: ignore # natural base
    df2out = dfin[columns := ["Pt_No", "tripod_x", "tripod_z", "absorption"]].copy(deep=True)

    # 保存每个能量值下的数据
    df2out.to_csv(Path.joinpath(scan_folder, f"{name_folder}_all.csv"), sep=",", index=False)
    dfout.append(df2out)

    # 新建对应的文件夹以及获取对应的位置
    # 获取切片的引索序列
    scancuttoff_index_list = df2out[(df2out.loc[:, "Pt_No"] == df2out.loc[:, "Pt_No"].min())].index.tolist()
    # scancuttoff_index_list=dfin2[(dfin2.loc[:,'Pt_No']==dfin2.loc[:,'Pt_No'].max())].index.tolist()  # noqa: ERA001
    # print(len(scancuttoff_index_list))  # noqa: ERA001

    # finding global min and max intensity cheapo way for transmission
    vmin = df2out.loc[:, "absorption"].min()
    vmax = df2out.loc[:, "absorption"].max()
    # print(vmin, vmax)  # noqa: ERA001

    for i in trange(len(scancuttoff_index_list) - 1):
        scan_name = f"{name_folder}_{i + 1:04d}"
        path_file_folder = create_folders(scan_folder, name_folder)

        a = scancuttoff_index_list[i]
        b = scancuttoff_index_list[i + 1]

        # 输出每个能量下的每次扫描数据，并画图
        df2out.iloc[a:b, :].to_csv(
            Path.joinpath(path_file_folder, f"{scan_name}.csv"), sep=",", index=True, header=True
        )
        slice_energy = df2out.iloc[a:b, :].pivot(index="tripod_z", columns="tripod_x", values="absorption")
        slice_energy.to_csv(Path.joinpath(path_file_folder, f"{scan_name}_pivot.csv"), sep=",", index=True, header=True)
        figure = data2map(slice_energy, scan_name, todisplay=False, vmin=vmin, vmax=vmax)
        figure.savefig(
            Path.joinpath(path_file_folder, f"{scan_name}.tif"),
            transparent=False,
            pad_inches=0.05,
            bbox_inches="tight",
            dpi=300,
            pil_kwargs={"compression": "tiff_lzw"},
        )
        plt.close()

#### 数据的清洗

In [ ]:
# 裁剪所有 absorption 列到最短长度
min_len = min(len(df) for df in dfout)
Mesh_data = np.array([df.iloc[:min_len, -1].values for df in dfout]).T

# 裁剪前3列位置信息并合并
Mesh_data = pd.concat([dfout[0].iloc[:min_len, :3].reset_index(drop=True), pd.DataFrame(Mesh_data)], axis=1).copy()
Mesh_data.to_csv(
    Path.joinpath(path_out, "Mesh_Absorption_data.csv"),
    sep=",",
    index=False,
    header=[
        r"Pt_No",
        r"tripod_x",
        r"tripod_z",
        r"absorption1",
        r"absorption2",
        r"absorption3",
        r"absorption4",
        r"absorption5",
    ],  # noqa: E501
)

#### PreEdge 和 PostEdge 的拟合

In [ ]:
from scipy.optimize import minimize

# 数据
E_Mn = np.array([6533.0, 6553.2, 6557.5, 6561.5, 6610.8])
Ra_Mn = np.array([0.01, 1.95670, 1.19821, 1.03919, 1.04312])
Rb_Mn = np.array([0.01, 0.531024, 1.04128, 1.42149, 0.983545])
constants = [Ra_Mn, Rb_Mn]


# 拟合函数
def model(params, energy, constants):
    eff, deltamu, pre_slope, pre_intercept = params
    Ra, Rb = constants  # noqa: N806
    return deltamu * (Ra * eff + Rb * (1 - eff)) + pre_slope * energy + pre_intercept


# 目标函数
def objective(params, energy, constants, data):
    pred = model(params, energy, constants)
    mse = np.mean((data - pred) ** 2)
    return mse


# 初始化参数
initial_guess = [0.5, 1.0, 0.0, 0.0]
bounds = [(0, 1), (0, np.inf), (-np.inf, np.inf), (-np.inf, np.inf)]

# 初始化结果容器
params_result = pd.DataFrame(index=Mesh_data.index, columns=["eff", "deltamu", "pre_slope", "pre_intercept"])  # type: ignore
absroption_result = pd.DataFrame(index=Mesh_data.index, columns=[f"Normal_{i}" for i in range(Mesh_data.shape[1] - 3)])  # type: ignore

# 拟合过程
for i in tqdm(Mesh_data.index):  # type: ignore
    temp = Mesh_data.iloc[i, 3:].values  # type: ignore
    result = minimize(
        objective, initial_guess, args=(E_Mn, constants, temp), bounds=bounds, method="SLSQP", options={"maxiter": 1000}
    )

    if result.success:
        eff, deltamu, pre_slope, pre_intercept = result.x
        params_result.loc[i] = result.x
        fitted_absorption = deltamu * (Ra_Mn * eff + Rb_Mn * (1 - eff))
        absroption_result.loc[i] = fitted_absorption

#### 验证结果的准确度

In [ ]:
# 保存数据
pd.concat([Mesh_data, absroption_result], axis=1, ignore_index=True).to_csv(  # type: ignore
    Path.joinpath(path_out, "Mesh_Absorption_results.csv"),
    sep=",",
    index=False,
    header=[
        r"Pt_No",
        r"tripod_x",
        r"tripod_z",
        r"absorption1",
        r"absorption2",
        r"absorption3",
        r"absorption4",
        r"absorption5",
        r"Normal1",
        r"Normal2",
        r"Normal3",
        r"Normal4",
        r"Normal5",
    ],  # noqa: E501
)
pd.concat([Mesh_data.iloc[:, :3], params_result], axis=1, ignore_index=True).to_csv(  # type: ignore
    Path.joinpath(path_out, "Mesh_Absorption_params.csv"),
    sep=",",
    index=False,
    header=[r"Pt_No", r"tripod_x", r"tripod_z", r"eff", r"deltamu", r"pre_slope", r"pre_intercept"],  # noqa: E501
)

#### 保存成 tiff 图片

In [ ]:
import tifffile

absroption_result_copy = pd.concat([Mesh_data.iloc[:, :3], absroption_result], axis=1, ignore_index=True)  # type: ignore
absroption_result_copy = absroption_result_copy.apply(pd.to_numeric, errors="coerce")


# 每像素 0.5 μm -> 20,000 pixels per cm
resolution = (20000 * 0.32, 10000 * 0.67)  # (x_res, y_res)

scan_folder = create_folders(path_out, "1_Results")
folder_names = [f.parts[-1][:-4] for f in path_master_file]

for idx in trange(3, absroption_result_copy.shape[1]):
    current_folder_name = folder_names[idx - 3]
    path_file_folder = create_folders(scan_folder, current_folder_name)

    columns = absroption_result_copy.columns
    temp = absroption_result_copy.loc[:, [columns[0], columns[1], columns[2], columns[idx]]]
    temp.columns = ["Pt_No", "tripod_x", "tripod_z", "absorption"]

    scancutoff_indices = temp[temp["Pt_No"] == temp["Pt_No"].min()].index.tolist()

    vmin = temp["absorption"].min(skipna=True)
    vmax = temp["absorption"].max(skipna=True)
    fig_list = []
    for i in trange(len(scancutoff_indices) - 1):
        scan_name = f"{current_folder_name}_{i + 1:04d}"
        a, b = scancutoff_indices[i], scancutoff_indices[i + 1]

        slice_df = temp.iloc[a:b, :]
        slice_df.to_csv(Path.joinpath(path_file_folder, f"{scan_name}.csv"), sep=",", index=True, header=True)

        pivot = slice_df.pivot(index="tripod_z", columns="tripod_x", values="absorption")
        pivot.to_csv(Path.joinpath(path_file_folder, f"{scan_name}_pivot.csv"), sep=",", index=True, header=True)
        fig_list.append(pivot.values)
        fig = data2map(pivot, scan_name, todisplay=False, vmin=vmin, vmax=vmax)
        fig.savefig(
            Path.joinpath(path_file_folder, f"{scan_name}.tif"),
            transparent=False,
            pad_inches=0.05,
            bbox_inches="tight",
            dpi=300,
            pil_kwargs={"compression": "tiff_lzw"},
        )
        plt.close()

        tifffile.imwrite(
            Path.joinpath(scan_folder, f"{path_file_folder}.tiff"),
            np.stack(fig_list, axis=0),
            photometric="minisblack",
            compression="lzw",
            resolution=resolution,
            resolutionunit="CENTIMETER",  # or "INCH"
        )

### Mn3, a+b=1

#### 读取数据

In [ ]:
# to read all master files
path_master_file = list(
    Path(
        r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XAS\Operando\αMnO2\MeshMapping\2023-CLAESS\Data\case2_1stDischarge\DownCell_Mn_2\Mn"  # noqa: E501, RUF001
    ).glob("*.dat")
)
header = "Pt_No tripod_z tripod_x energyc a_i0i1_timer a_i0_1 a_i0_2 a_i1_1 a_i1_2 n_timer n_i0_1 n_i0_2 n_i1_1 n_i1_2 n_i2_1 n_i2_2 n_icr n_tcr n_sca1 n_sca2 n_sca3 n_sca4 sr_i_1 dt"  # noqa: E501
headers = header.split(" ")

In [ ]:
# 处理 mesh 数据
dfout = []
scan_folder = create_folders(path_out, "0_PureAbsorption")
# read data files and clean it
for master_file_name in tqdm(path_master_file):
    name_folder = master_file_name.parts[-1][:-4]

    dfin = pd.read_csv(master_file_name, index_col=None, header=None, names=headers, sep=r"\s+", comment="#")

    NaNindex = dfin.isnull().any(axis=1).to_numpy().nonzero()[0].tolist()
    dfin.drop(NaNindex, axis=0, inplace=True)

    # 计算 Absorption
    i0 = dfin.loc[:, ["a_i0_1", "a_i0_2"]].sum(axis=1).values
    i1 = dfin.loc[:, ["a_i1_1", "a_i1_2"]].sum(axis=1).values
    dfin["absorption"] = np.log(i0 / i1)  # type: ignore # natural base
    df2out = dfin[columns := ["Pt_No", "tripod_x", "tripod_z", "absorption"]].copy(deep=True)

    # 保存每个能量值下的数据
    df2out.to_csv(Path.joinpath(scan_folder, f"{name_folder}_all.csv"), sep=",", index=False)
    dfout.append(df2out)

    # 新建对应的文件夹以及获取对应的位置
    # 获取切片的引索序列
    scancuttoff_index_list = df2out[(df2out.loc[:, "Pt_No"] == df2out.loc[:, "Pt_No"].min())].index.tolist()
    # scancuttoff_index_list=dfin2[(dfin2.loc[:,'Pt_No']==dfin2.loc[:,'Pt_No'].max())].index.tolist()  # noqa: ERA001
    # print(len(scancuttoff_index_list))  # noqa: ERA001

    # finding global min and max intensity cheapo way for transmission
    vmin = df2out.loc[:, "absorption"].min()
    vmax = df2out.loc[:, "absorption"].max()
    # print(vmin, vmax)  # noqa: ERA001

    for i in trange(len(scancuttoff_index_list) - 1):
        scan_name = f"{name_folder}_{i + 1:04d}"
        path_file_folder = create_folders(scan_folder, name_folder)

        a = scancuttoff_index_list[i]
        b = scancuttoff_index_list[i + 1]

        # 输出每个能量下的每次扫描数据，并画图
        df2out.iloc[a:b, :].to_csv(
            Path.joinpath(path_file_folder, f"{scan_name}.csv"), sep=",", index=True, header=True
        )
        slice_energy = df2out.iloc[a:b, :].pivot(index="tripod_z", columns="tripod_x", values="absorption")
        slice_energy.to_csv(Path.joinpath(path_file_folder, f"{scan_name}_pivot.csv"), sep=",", index=True, header=True)
        figure = data2map(slice_energy, scan_name, todisplay=False, vmin=vmin, vmax=vmax)
        figure.savefig(
            Path.joinpath(path_file_folder, f"{scan_name}.tif"),
            transparent=False,
            pad_inches=0.05,
            bbox_inches="tight",
            dpi=300,
            pil_kwargs={"compression": "tiff_lzw"},
        )
        plt.close()

#### 数据的清洗

In [ ]:
# 裁剪所有 absorption 列到最短长度
min_len = min(len(df) for df in dfout)
Mesh_data = np.array([df.iloc[:min_len, -1].values for df in dfout]).T

# 裁剪前3列位置信息并合并
Mesh_data = pd.concat([dfout[0].iloc[:min_len, :3].reset_index(drop=True), pd.DataFrame(Mesh_data)], axis=1).copy()
Mesh_data.to_csv(
    Path.joinpath(path_out, "Mesh_Absorption_data.csv"),
    sep=",",
    index=False,
    header=[
        r"Pt_No",
        r"tripod_x",
        r"tripod_z",
        r"absorption1",
        r"absorption2",
        r"absorption3",
        r"absorption4",
        r"absorption5",
    ],  # noqa: E501
)

#### PreEdge 和 PostEdge 的拟合

In [ ]:
from scipy.optimize import minimize

# 数据
E_Mn = np.array([6533.0, 6553.2, 6557.5, 6561.5, 6610.8])
Ra_Mn = np.array([0.01, 1.95670, 1.19821, 1.03919, 1.04312])
Rb_Mn = np.array([0.01, 0.531024, 1.04128, 1.42149, 0.983545])
constants = [Ra_Mn, Rb_Mn]


# 拟合函数
def model(params, energy, constants):
    eff, deltamu, pre_slope, pre_intercept = params
    Ra, Rb = constants  # noqa: N806
    return deltamu * (Ra * eff + Rb * (1 - eff)) + pre_slope * energy + pre_intercept


# 目标函数
def objective(params, energy, constants, data):
    pred = model(params, energy, constants)
    mse = np.mean((data - pred) ** 2)
    return mse


# 初始化参数
initial_guess = [0.5, 1.0, 0.0, 0.0]
bounds = [(0, 1), (0, np.inf), (-np.inf, np.inf), (-np.inf, np.inf)]

# 初始化结果容器
params_result = pd.DataFrame(index=Mesh_data.index, columns=["eff", "deltamu", "pre_slope", "pre_intercept"])  # type: ignore
absroption_result = pd.DataFrame(index=Mesh_data.index, columns=[f"Normal_{i}" for i in range(Mesh_data.shape[1] - 3)])  # type: ignore

# 拟合过程
for i in tqdm(Mesh_data.index):  # type: ignore
    temp = Mesh_data.iloc[i, 3:].values  # type: ignore
    result = minimize(
        objective, initial_guess, args=(E_Mn, constants, temp), bounds=bounds, method="SLSQP", options={"maxiter": 1000}
    )

    if result.success:
        eff, deltamu, pre_slope, pre_intercept = result.x
        params_result.loc[i] = result.x
        fitted_absorption = deltamu * (Ra_Mn * eff + Rb_Mn * (1 - eff))
        absroption_result.loc[i] = fitted_absorption

#### 验证结果的准确度

In [ ]:
# 保存数据
pd.concat([Mesh_data, absroption_result], axis=1, ignore_index=True).to_csv(  # type: ignore
    Path.joinpath(path_out, "Mesh_Absorption_results.csv"),
    sep=",",
    index=False,
    header=[
        r"Pt_No",
        r"tripod_x",
        r"tripod_z",
        r"absorption1",
        r"absorption2",
        r"absorption3",
        r"absorption4",
        r"absorption5",
        r"Normal1",
        r"Normal2",
        r"Normal3",
        r"Normal4",
        r"Normal5",
    ],  # noqa: E501
)
pd.concat([Mesh_data.iloc[:, :3], params_result], axis=1, ignore_index=True).to_csv(  # type: ignore
    Path.joinpath(path_out, "Mesh_Absorption_params.csv"),
    sep=",",
    index=False,
    header=[r"Pt_No", r"tripod_x", r"tripod_z", r"eff", r"deltamu", r"pre_slope", r"pre_intercept"],  # noqa: E501
)

#### 保存成 tiff 图片

In [ ]:
import tifffile

absroption_result_copy = pd.concat([Mesh_data.iloc[:, :3], absroption_result], axis=1, ignore_index=True)  # type: ignore
absroption_result_copy = absroption_result_copy.apply(pd.to_numeric, errors="coerce")


# 每像素 0.5 μm -> 20,000 pixels per cm
resolution = (20000 * 0.32, 10000 * 0.67)  # (x_res, y_res)

scan_folder = create_folders(path_out, "1_Results")
folder_names = [f.parts[-1][:-4] for f in path_master_file]

for idx in trange(3, absroption_result_copy.shape[1]):
    current_folder_name = folder_names[idx - 3]
    path_file_folder = create_folders(scan_folder, current_folder_name)

    columns = absroption_result_copy.columns
    temp = absroption_result_copy.loc[:, [columns[0], columns[1], columns[2], columns[idx]]]
    temp.columns = ["Pt_No", "tripod_x", "tripod_z", "absorption"]

    scancutoff_indices = temp[temp["Pt_No"] == temp["Pt_No"].min()].index.tolist()

    vmin = temp["absorption"].min(skipna=True)
    vmax = temp["absorption"].max(skipna=True)
    fig_list = []
    for i in trange(len(scancutoff_indices) - 1):
        scan_name = f"{current_folder_name}_{i + 1:04d}"
        a, b = scancutoff_indices[i], scancutoff_indices[i + 1]

        slice_df = temp.iloc[a:b, :]
        slice_df.to_csv(Path.joinpath(path_file_folder, f"{scan_name}.csv"), sep=",", index=True, header=True)

        pivot = slice_df.pivot(index="tripod_z", columns="tripod_x", values="absorption")
        pivot.to_csv(Path.joinpath(path_file_folder, f"{scan_name}_pivot.csv"), sep=",", index=True, header=True)
        fig_list.append(pivot.values)
        fig = data2map(pivot, scan_name, todisplay=False, vmin=vmin, vmax=vmax)
        fig.savefig(
            Path.joinpath(path_file_folder, f"{scan_name}.tif"),
            transparent=False,
            pad_inches=0.05,
            bbox_inches="tight",
            dpi=300,
            pil_kwargs={"compression": "tiff_lzw"},
        )
        plt.close()

        tifffile.imwrite(
            Path.joinpath(scan_folder, f"{path_file_folder}.tiff"),
            np.stack(fig_list, axis=0),
            photometric="minisblack",
            compression="lzw",
            resolution=resolution,
            resolutionunit="CENTIMETER",  # or "INCH"
        )

### Mn2, a+b=1

#### 读取数据

In [ ]:
# to read all master files
path_master_file = list(
    Path(
        r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XAS\Operando\αMnO2\MeshMapping\2023-CLAESS\Data\case1_1stCharge_1stDischarge\UpCell_MnFree\Mn"  # noqa: E501, RUF001
    ).glob("*.dat")
)
header = "Pt_No tripod_z tripod_x energyc a_i0i1_timer a_i0_1 a_i0_2 a_i1_1 a_i1_2 n_timer n_i0_1 n_i0_2 n_i1_1 n_i1_2 n_i2_1 n_i2_2 n_icr n_tcr n_sca1 n_sca2 n_sca3 n_sca4 sr_i_1 dt"  # noqa: E501
headers = header.split(" ")

In [ ]:
# 处理 mesh 数据
dfout = []
scan_folder = create_folders(path_out, "0_PureAbsorption")
# read data files and clean it
for master_file_name in tqdm(path_master_file):
    name_folder = master_file_name.parts[-1][:-4]

    dfin = pd.read_csv(master_file_name, index_col=None, header=None, names=headers, sep=r"\s+", comment="#")

    NaNindex = dfin.isnull().any(axis=1).to_numpy().nonzero()[0].tolist()
    dfin.drop(NaNindex, axis=0, inplace=True)

    # 计算 Absorption
    i0 = dfin.loc[:, ["a_i0_1", "a_i0_2"]].sum(axis=1).values
    i1 = dfin.loc[:, ["a_i1_1", "a_i1_2"]].sum(axis=1).values
    dfin["absorption"] = np.log(i0 / i1)  # type: ignore # natural base
    df2out = dfin[columns := ["Pt_No", "tripod_x", "tripod_z", "absorption"]].copy(deep=True)

    # 保存每个能量值下的数据
    df2out.to_csv(Path.joinpath(scan_folder, f"{name_folder}_all.csv"), sep=",", index=False)
    dfout.append(df2out)

    # 新建对应的文件夹以及获取对应的位置
    # 获取切片的引索序列
    scancuttoff_index_list = df2out[(df2out.loc[:, "Pt_No"] == df2out.loc[:, "Pt_No"].min())].index.tolist()
    # scancuttoff_index_list=dfin2[(dfin2.loc[:,'Pt_No']==dfin2.loc[:,'Pt_No'].max())].index.tolist()  # noqa: ERA001
    # print(len(scancuttoff_index_list))  # noqa: ERA001

    # finding global min and max intensity cheapo way for transmission
    vmin = df2out.loc[:, "absorption"].min()
    vmax = df2out.loc[:, "absorption"].max()
    # print(vmin, vmax)  # noqa: ERA001

    for i in trange(len(scancuttoff_index_list) - 1):
        scan_name = f"{name_folder}_{i + 1:04d}"
        path_file_folder = create_folders(scan_folder, name_folder)

        a = scancuttoff_index_list[i]
        b = scancuttoff_index_list[i + 1]

        # 输出每个能量下的每次扫描数据，并画图
        df2out.iloc[a:b, :].to_csv(
            Path.joinpath(path_file_folder, f"{scan_name}.csv"), sep=",", index=True, header=True
        )
        slice_energy = df2out.iloc[a:b, :].pivot(index="tripod_z", columns="tripod_x", values="absorption")
        slice_energy.to_csv(Path.joinpath(path_file_folder, f"{scan_name}_pivot.csv"), sep=",", index=True, header=True)
        figure = data2map(slice_energy, scan_name, todisplay=False, vmin=vmin, vmax=vmax)
        figure.savefig(
            Path.joinpath(path_file_folder, f"{scan_name}.tif"),
            transparent=False,
            pad_inches=0.05,
            bbox_inches="tight",
            dpi=300,
            pil_kwargs={"compression": "tiff_lzw"},
        )
        plt.close()

#### 数据的清洗

In [ ]:
# 裁剪所有 absorption 列到最短长度
min_len = min(len(df) for df in dfout)
Mesh_data = np.array([df.iloc[:min_len, -1].values for df in dfout]).T

# 裁剪前3列位置信息并合并
Mesh_data = pd.concat([dfout[0].iloc[:min_len, :3].reset_index(drop=True), pd.DataFrame(Mesh_data)], axis=1).copy()
Mesh_data.to_csv(
    Path.joinpath(path_out, "Mesh_Absorption_data.csv"),
    sep=",",
    index=False,
    header=[
        r"Pt_No",
        r"tripod_x",
        r"tripod_z",
        r"absorption1",
        r"absorption2",
        r"absorption3",
        r"absorption4",
        r"absorption5",
    ],  # noqa: E501
)

#### PreEdge 和 PostEdge 的拟合

In [ ]:
from scipy.optimize import minimize

# 数据
E_Mn = np.array([6533.0, 6553.2, 6557.5, 6561.5, 6610.8])
Ra_Mn = np.array([0.01, 1.95670, 1.19821, 1.03919, 1.04312])
Rb_Mn = np.array([0.01, 0.531024, 1.04128, 1.42149, 0.983545])
constants = [Ra_Mn, Rb_Mn]


# 拟合函数
def model(params, energy, constants):
    eff, deltamu, pre_slope, pre_intercept = params
    Ra, Rb = constants  # noqa: N806
    return deltamu * (Ra * eff + Rb * (1 - eff)) + pre_slope * energy + pre_intercept


# 目标函数
def objective(params, energy, constants, data):
    pred = model(params, energy, constants)
    mse = np.mean((data - pred) ** 2)
    return mse


# 初始化参数
initial_guess = [0.5, 1.0, 0.0, 0.0]
bounds = [(0, 1), (0, np.inf), (-np.inf, np.inf), (-np.inf, np.inf)]

# 初始化结果容器
params_result = pd.DataFrame(index=Mesh_data.index, columns=["eff", "deltamu", "pre_slope", "pre_intercept"])  # type: ignore
absroption_result = pd.DataFrame(index=Mesh_data.index, columns=[f"Normal_{i}" for i in range(Mesh_data.shape[1] - 3)])  # type: ignore

# 拟合过程
for i in tqdm(Mesh_data.index):  # type: ignore
    temp = Mesh_data.iloc[i, 3:].values  # type: ignore
    result = minimize(
        objective, initial_guess, args=(E_Mn, constants, temp), bounds=bounds, method="SLSQP", options={"maxiter": 1000}
    )

    if result.success:
        eff, deltamu, pre_slope, pre_intercept = result.x
        params_result.loc[i] = result.x
        fitted_absorption = deltamu * (Ra_Mn * eff + Rb_Mn * (1 - eff))
        absroption_result.loc[i] = fitted_absorption

#### 验证结果的准确度

In [ ]:
# 保存数据
pd.concat([Mesh_data, absroption_result], axis=1, ignore_index=True).to_csv(  # type: ignore
    Path.joinpath(path_out, "Mesh_Absorption_results.csv"),
    sep=",",
    index=False,
    header=[
        r"Pt_No",
        r"tripod_x",
        r"tripod_z",
        r"absorption1",
        r"absorption2",
        r"absorption3",
        r"absorption4",
        r"absorption5",
        r"Normal1",
        r"Normal2",
        r"Normal3",
        r"Normal4",
        r"Normal5",
    ],  # noqa: E501
)
pd.concat([Mesh_data.iloc[:, :3], params_result], axis=1, ignore_index=True).to_csv(  # type: ignore
    Path.joinpath(path_out, "Mesh_Absorption_params.csv"),
    sep=",",
    index=False,
    header=[r"Pt_No", r"tripod_x", r"tripod_z", r"eff", r"deltamu", r"pre_slope", r"pre_intercept"],  # noqa: E501
)

#### 保存成 tiff 图片

In [ ]:
import tifffile

absroption_result_copy = pd.concat([Mesh_data.iloc[:, :3], absroption_result], axis=1, ignore_index=True)  # type: ignore
absroption_result_copy = absroption_result_copy.apply(pd.to_numeric, errors="coerce")


# 每像素 0.5 μm -> 20,000 pixels per cm
resolution = (20000 * 0.32, 10000 * 0.67)  # (x_res, y_res)

scan_folder = create_folders(path_out, "1_Results")
folder_names = [f.parts[-1][:-4] for f in path_master_file]

for idx in trange(3, absroption_result_copy.shape[1]):
    current_folder_name = folder_names[idx - 3]
    path_file_folder = create_folders(scan_folder, current_folder_name)

    columns = absroption_result_copy.columns
    temp = absroption_result_copy.loc[:, [columns[0], columns[1], columns[2], columns[idx]]]
    temp.columns = ["Pt_No", "tripod_x", "tripod_z", "absorption"]

    scancutoff_indices = temp[temp["Pt_No"] == temp["Pt_No"].min()].index.tolist()

    vmin = temp["absorption"].min(skipna=True)
    vmax = temp["absorption"].max(skipna=True)
    fig_list = []
    for i in trange(len(scancutoff_indices) - 1):
        scan_name = f"{current_folder_name}_{i + 1:04d}"
        a, b = scancutoff_indices[i], scancutoff_indices[i + 1]

        slice_df = temp.iloc[a:b, :]
        slice_df.to_csv(Path.joinpath(path_file_folder, f"{scan_name}.csv"), sep=",", index=True, header=True)

        pivot = slice_df.pivot(index="tripod_z", columns="tripod_x", values="absorption")
        pivot.to_csv(Path.joinpath(path_file_folder, f"{scan_name}_pivot.csv"), sep=",", index=True, header=True)
        fig_list.append(pivot.values)
        fig = data2map(pivot, scan_name, todisplay=False, vmin=vmin, vmax=vmax)
        fig.savefig(
            Path.joinpath(path_file_folder, f"{scan_name}.tif"),
            transparent=False,
            pad_inches=0.05,
            bbox_inches="tight",
            dpi=300,
            pil_kwargs={"compression": "tiff_lzw"},
        )
        plt.close()

        tifffile.imwrite(
            Path.joinpath(scan_folder, f"{path_file_folder}.tiff"),
            np.stack(fig_list, axis=0),
            photometric="minisblack",
            compression="lzw",
            resolution=resolution,
            resolutionunit="CENTIMETER",  # or "INCH"
        )

### Mn1, a+b = 1

#### 读取数据

In [ ]:
# to read all master files
path_master_file = list(
    Path(
        r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XAS\Operando\αMnO2\MeshMapping\2023-CLAESS\Data\case1_1stCharge_1stDischarge\DownCell_Mn\Mn"  # noqa: E501, RUF001
    ).glob("*.dat")
)
header = "Pt_No tripod_z tripod_x energyc a_i0i1_timer a_i0_1 a_i0_2 a_i1_1 a_i1_2 n_timer n_i0_1 n_i0_2 n_i1_1 n_i1_2 n_i2_1 n_i2_2 n_icr n_tcr n_sca1 n_sca2 n_sca3 n_sca4 sr_i_1 dt"  # noqa: E501
headers = header.split(" ")

In [ ]:
# 处理 mesh 数据
dfout = []
scan_folder = create_folders(path_out, "0_PureAbsorption")
# read data files and clean it
for master_file_name in tqdm(path_master_file):
    name_folder = master_file_name.parts[-1][:-4]

    dfin = pd.read_csv(master_file_name, index_col=None, header=None, names=headers, sep=r"\s+", comment="#")

    NaNindex = dfin.isnull().any(axis=1).to_numpy().nonzero()[0].tolist()
    dfin.drop(NaNindex, axis=0, inplace=True)

    # 计算 Absorption
    i0 = dfin.loc[:, ["a_i0_1", "a_i0_2"]].sum(axis=1).values
    i1 = dfin.loc[:, ["a_i1_1", "a_i1_2"]].sum(axis=1).values
    dfin["absorption"] = np.log(i0 / i1)  # type: ignore # natural base
    df2out = dfin[columns := ["Pt_No", "tripod_x", "tripod_z", "absorption"]].copy(deep=True)

    # 保存每个能量值下的数据
    df2out.to_csv(Path.joinpath(scan_folder, f"{name_folder}_all.csv"), sep=",", index=False)
    dfout.append(df2out)

    # 新建对应的文件夹以及获取对应的位置
    # 获取切片的引索序列
    scancuttoff_index_list = df2out[(df2out.loc[:, "Pt_No"] == df2out.loc[:, "Pt_No"].min())].index.tolist()
    # scancuttoff_index_list=dfin2[(dfin2.loc[:,'Pt_No']==dfin2.loc[:,'Pt_No'].max())].index.tolist()  # noqa: ERA001
    # print(len(scancuttoff_index_list))  # noqa: ERA001

    # finding global min and max intensity cheapo way for transmission
    vmin = df2out.loc[:, "absorption"].min()
    vmax = df2out.loc[:, "absorption"].max()
    # print(vmin, vmax)  # noqa: ERA001

    for i in trange(len(scancuttoff_index_list) - 1):
        scan_name = f"{name_folder}_{i + 1:04d}"
        path_file_folder = create_folders(scan_folder, name_folder)

        a = scancuttoff_index_list[i]
        b = scancuttoff_index_list[i + 1]

        # 输出每个能量下的每次扫描数据，并画图
        df2out.iloc[a:b, :].to_csv(
            Path.joinpath(path_file_folder, f"{scan_name}.csv"), sep=",", index=True, header=True
        )
        slice_energy = df2out.iloc[a:b, :].pivot(index="tripod_z", columns="tripod_x", values="absorption")
        slice_energy.to_csv(Path.joinpath(path_file_folder, f"{scan_name}_pivot.csv"), sep=",", index=True, header=True)
        figure = data2map(slice_energy, scan_name, todisplay=False, vmin=vmin, vmax=vmax)
        figure.savefig(
            Path.joinpath(path_file_folder, f"{scan_name}.tif"),
            transparent=False,
            pad_inches=0.05,
            bbox_inches="tight",
            dpi=300,
            pil_kwargs={"compression": "tiff_lzw"},
        )
        plt.close()

#### 数据的清洗

In [ ]:
# 裁剪所有 absorption 列到最短长度
min_len = min(len(df) for df in dfout)
Mesh_data = np.array([df.iloc[:min_len, -1].values for df in dfout]).T

# 裁剪前3列位置信息并合并
Mesh_data = pd.concat([dfout[0].iloc[:min_len, :3].reset_index(drop=True), pd.DataFrame(Mesh_data)], axis=1).copy()
Mesh_data.to_csv(
    Path.joinpath(path_out, "Mesh_Absorption_data.csv"),
    sep=",",
    index=False,
    header=[
        r"Pt_No",
        r"tripod_x",
        r"tripod_z",
        r"absorption1",
        r"absorption2",
        r"absorption3",
        r"absorption4",
        r"absorption5",
    ],  # noqa: E501
)

#### PreEdge 和 PostEdge 的拟合

In [ ]:
from scipy.optimize import minimize

# 数据
E_Mn = np.array([6533.0, 6553.2, 6557.5, 6561.5, 6610.8])
Ra_Mn = np.array([0.01, 1.95670, 1.19821, 1.03919, 1.04312])
Rb_Mn = np.array([0.01, 0.531024, 1.04128, 1.42149, 0.983545])
constants = [Ra_Mn, Rb_Mn]

# 拟合函数
def model(params, energy, constants):
    eff, deltamu, pre_slope, pre_intercept = params
    Ra, Rb = constants  # noqa: N806
    return deltamu * (Ra * eff + Rb * (1 - eff)) + pre_slope * energy + pre_intercept


# 目标函数
def objective(params, energy, constants, data):
    pred = model(params, energy, constants)
    mse = np.mean((data - pred) ** 2)
    return mse


# 初始化参数
initial_guess = [0.5, 1.0, 0.0, 0.0]
bounds = [(0, 1), (0, np.inf), (-np.inf, np.inf), (-np.inf, np.inf)]

# 初始化结果容器
params_result = pd.DataFrame(index=Mesh_data.index, columns=["eff", "deltamu", "pre_slope", "pre_intercept"])  # type: ignore
absroption_result = pd.DataFrame(index=Mesh_data.index, columns=[f"Normal_{i}" for i in range(Mesh_data.shape[1] - 3)])  # type: ignore
Mn2 = pd.DataFrame(index=Mesh_data.index, columns=[f"Mn2+_{i}" for i in range(Mesh_data.shape[1] - 3)])  # type: ignore
MnO2 = pd.DataFrame(index=Mesh_data.index, columns=[f"MnO2_{i}" for i in range(Mesh_data.shape[1] - 3)])  # type: ignore

# 拟合过程
for i in tqdm(Mesh_data.index):  # type: ignore
    temp = Mesh_data.iloc[i, 3:].values  # type: ignore
    result = minimize(
        objective, initial_guess, args=(E_Mn, constants, temp), bounds=bounds, method="SLSQP", options={"maxiter": 1000}
    )

    if result.success:
        eff, deltamu, pre_slope, pre_intercept = result.x
        params_result.loc[i] = result.x
        fitted_absorption = deltamu * (Ra_Mn * eff + Rb_Mn * (1 - eff))
        absroption_result.loc[i] = fitted_absorption
        Mn2.loc[i] = deltamu * Ra_Mn * eff
        MnO2.loc[i] = deltamu * Rb_Mn * (1 - eff)

#### 验证结果的准确度

In [ ]:
# 保存数据
pd.concat([Mesh_data, absroption_result], axis=1, ignore_index=True).to_csv(  # type: ignore
    Path.joinpath(path_out, "Mesh_Absorption_results.csv"),
    sep=",",
    index=False,
    header=[
        r"Pt_No",
        r"tripod_x",
        r"tripod_z",
        r"absorption1",
        r"absorption2",
        r"absorption3",
        r"absorption4",
        r"absorption5",
        r"Normal1",
        r"Normal2",
        r"Normal3",
        r"Normal4",
        r"Normal5",
    ],  # noqa: E501
)
pd.concat([Mesh_data.iloc[:, :3], params_result], axis=1, ignore_index=True).to_csv(  # type: ignore
    Path.joinpath(path_out, "Mesh_Absorption_params.csv"),
    sep=",",
    index=False,
    header=[r"Pt_No", r"tripod_x", r"tripod_z", r"eff", r"deltamu", r"pre_slope", r"pre_intercept"],  # noqa: E501
)

In [ ]:
# 保存数据
pd.concat([Mesh_data, Mn2], axis=1, ignore_index=True).to_csv(  # type: ignore
    Path.joinpath(path_out, "Mesh_Absorption_results_Mn2.csv"),
    sep=",",
    index=False,
    header=[
        r"Pt_No",
        r"tripod_x",
        r"tripod_z",
        r"absorption1",
        r"absorption2",
        r"absorption3",
        r"absorption4",
        r"absorption5",
        r"Mn_2_1",
        r"Mn_2_2",
        r"Mn_2_3",
        r"Mn_2_4",
        r"Mn_2_5",
    ],  # noqa: E501
)
# 保存数据
pd.concat([Mesh_data, MnO2], axis=1, ignore_index=True).to_csv(  # type: ignore
    Path.joinpath(path_out, "Mesh_Absorption_results_MnO2.csv"),
    sep=",",
    index=False,
    header=[
        r"Pt_No",
        r"tripod_x",
        r"tripod_z",
        r"absorption1",
        r"absorption2",
        r"absorption3",
        r"absorption4",
        r"absorption5",
        r"MnO2_1",
        r"MnO2_2",
        r"MnO2_3",
        r"MnO2_4",
        r"MnO2_5",
    ],  # noqa: E501
)

#### 保存成 tiff 图片

In [ ]:
import tifffile

absroption_result_copy = pd.concat([Mesh_data.iloc[:, :3], absroption_result], axis=1, ignore_index=True)  # type: ignore
absroption_result_copy = absroption_result_copy.apply(pd.to_numeric, errors="coerce")


# 每像素 0.5 μm -> 20,000 pixels per cm
resolution = (20000 * 0.32, 10000 * 0.67)  # (x_res, y_res)

scan_folder = create_folders(path_out, "1_Results")
folder_names = [f.parts[-1][:-4] for f in path_master_file]

for idx in trange(3, absroption_result_copy.shape[1]):
    current_folder_name = folder_names[idx - 3]
    path_file_folder = create_folders(scan_folder, current_folder_name)

    columns = absroption_result_copy.columns
    temp = absroption_result_copy.loc[:, [columns[0], columns[1], columns[2], columns[idx]]]
    temp.columns = ["Pt_No", "tripod_x", "tripod_z", "absorption"]

    scancutoff_indices = temp[temp["Pt_No"] == temp["Pt_No"].min()].index.tolist()

    vmin = temp["absorption"].min(skipna=True)
    vmax = temp["absorption"].max(skipna=True)
    fig_list = []
    for i in trange(len(scancutoff_indices) - 1):
        scan_name = f"{current_folder_name}_{i + 1:04d}"
        a, b = scancutoff_indices[i], scancutoff_indices[i + 1]

        slice_df = temp.iloc[a:b, :]
        slice_df.to_csv(Path.joinpath(path_file_folder, f"{scan_name}.csv"), sep=",", index=True, header=True)

        pivot = slice_df.pivot(index="tripod_z", columns="tripod_x", values="absorption")
        pivot.to_csv(Path.joinpath(path_file_folder, f"{scan_name}_pivot.csv"), sep=",", index=True, header=True)
        fig_list.append(pivot.values)
        fig = data2map(pivot, scan_name, todisplay=False, vmin=vmin, vmax=vmax)
        fig.savefig(
            Path.joinpath(path_file_folder, f"{scan_name}.tif"),
            transparent=False,
            pad_inches=0.05,
            bbox_inches="tight",
            dpi=300,
            pil_kwargs={"compression": "tiff_lzw"},
        )
        plt.close()

        tifffile.imwrite(
            Path.joinpath(scan_folder, f"{path_file_folder}.tiff"),
            np.stack(fig_list, axis=0),
            photometric="minisblack",
            compression="lzw",
            resolution=resolution,
            resolutionunit="CENTIMETER",  # or "INCH"
        )

In [ ]:
import tifffile

absroption_result_copy = pd.concat([Mesh_data.iloc[:, :3], Mn2], axis=1, ignore_index=True)  # type: ignore
absroption_result_copy = absroption_result_copy.apply(pd.to_numeric, errors="coerce")


# 每像素 0.5 μm -> 20,000 pixels per cm
resolution = (20000 * 0.32, 10000 * 0.67)  # (x_res, y_res)

scan_folder = create_folders(path_out, "1_Results_Mn2")
folder_names = [f.parts[-1][:-4] for f in path_master_file]

for idx in trange(3, absroption_result_copy.shape[1]):
    current_folder_name = folder_names[idx - 3]
    path_file_folder = create_folders(scan_folder, current_folder_name)

    columns = absroption_result_copy.columns
    temp = absroption_result_copy.loc[:, [columns[0], columns[1], columns[2], columns[idx]]]
    temp.columns = ["Pt_No", "tripod_x", "tripod_z", "absorption"]

    scancutoff_indices = temp[temp["Pt_No"] == temp["Pt_No"].min()].index.tolist()

    vmin = temp["absorption"].min(skipna=True)
    vmax = temp["absorption"].max(skipna=True)
    fig_list = []
    for i in trange(len(scancutoff_indices) - 1):
        scan_name = f"{current_folder_name}_{i + 1:04d}"
        a, b = scancutoff_indices[i], scancutoff_indices[i + 1]

        slice_df = temp.iloc[a:b, :]
        slice_df.to_csv(Path.joinpath(path_file_folder, f"{scan_name}.csv"), sep=",", index=True, header=True)

        pivot = slice_df.pivot(index="tripod_z", columns="tripod_x", values="absorption")
        pivot.to_csv(Path.joinpath(path_file_folder, f"{scan_name}_pivot.csv"), sep=",", index=True, header=True)
        fig_list.append(pivot.values)
        fig = data2map(pivot, scan_name, todisplay=False, vmin=vmin, vmax=vmax)
        fig.savefig(
            Path.joinpath(path_file_folder, f"{scan_name}.tif"),
            transparent=False,
            pad_inches=0.05,
            bbox_inches="tight",
            dpi=300,
            pil_kwargs={"compression": "tiff_lzw"},
        )
        plt.close()

        tifffile.imwrite(
            Path.joinpath(scan_folder, f"{path_file_folder}.tiff"),
            np.stack(fig_list, axis=0),
            photometric="minisblack",
            compression="lzw",
            resolution=resolution,
            resolutionunit="CENTIMETER",  # or "INCH"
        )

In [ ]:
import tifffile

absroption_result_copy = pd.concat([Mesh_data.iloc[:, :3], MnO2], axis=1, ignore_index=True)  # type: ignore
absroption_result_copy = absroption_result_copy.apply(pd.to_numeric, errors="coerce")


# 每像素 0.5 μm -> 20,000 pixels per cm
resolution = (20000 * 0.32, 10000 * 0.67)  # (x_res, y_res)

scan_folder = create_folders(path_out, "1_Results_MnO2")
folder_names = [f.parts[-1][:-4] for f in path_master_file]

for idx in trange(3, absroption_result_copy.shape[1]):
    current_folder_name = folder_names[idx - 3]
    path_file_folder = create_folders(scan_folder, current_folder_name)

    columns = absroption_result_copy.columns
    temp = absroption_result_copy.loc[:, [columns[0], columns[1], columns[2], columns[idx]]]
    temp.columns = ["Pt_No", "tripod_x", "tripod_z", "absorption"]

    scancutoff_indices = temp[temp["Pt_No"] == temp["Pt_No"].min()].index.tolist()

    vmin = temp["absorption"].min(skipna=True)
    vmax = temp["absorption"].max(skipna=True)
    fig_list = []
    for i in trange(len(scancutoff_indices) - 1):
        scan_name = f"{current_folder_name}_{i + 1:04d}"
        a, b = scancutoff_indices[i], scancutoff_indices[i + 1]

        slice_df = temp.iloc[a:b, :]
        slice_df.to_csv(Path.joinpath(path_file_folder, f"{scan_name}.csv"), sep=",", index=True, header=True)

        pivot = slice_df.pivot(index="tripod_z", columns="tripod_x", values="absorption")
        pivot.to_csv(Path.joinpath(path_file_folder, f"{scan_name}_pivot.csv"), sep=",", index=True, header=True)
        fig_list.append(pivot.values)
        fig = data2map(pivot, scan_name, todisplay=False, vmin=vmin, vmax=vmax)
        fig.savefig(
            Path.joinpath(path_file_folder, f"{scan_name}.tif"),
            transparent=False,
            pad_inches=0.05,
            bbox_inches="tight",
            dpi=300,
            pil_kwargs={"compression": "tiff_lzw"},
        )
        plt.close()

        tifffile.imwrite(
            Path.joinpath(scan_folder, f"{path_file_folder}.tiff"),
            np.stack(fig_list, axis=0),
            photometric="minisblack",
            compression="lzw",
            resolution=resolution,
            resolutionunit="CENTIMETER",  # or "INCH"
        )

### Zn4

#### 读取数据

In [ ]:
# to read all master files
path_master_file = list(
    Path(
        r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\PaperDos\XAS\Operando\αMnO2\MeshMapping\2023-CLAESS\Data\case2_1stDischarge\UpCell_MnFree_2\Zn"  # noqa: E501, RUF001
    ).glob("*.dat")
)
header = "Pt_No tripod_z tripod_x energyc a_i0i1_timer a_i0_1 a_i0_2 a_i1_1 a_i1_2 n_timer n_i0_1 n_i0_2 n_i1_1 n_i1_2 n_i2_1 n_i2_2 n_icr n_tcr n_sca1 n_sca2 n_sca3 n_sca4 sr_i_1 dt"  # noqa: E501
headers = header.split(" ")

In [ ]:
# 处理 mesh 数据
dfout = []
scan_folder = create_folders(path_out, "0_PureAbsorption")
# read data files and clean it
for master_file_name in tqdm(path_master_file):
    name_folder = master_file_name.parts[-1][:-4]

    dfin = pd.read_csv(master_file_name, index_col=None, header=None, names=headers, sep=r"\s+", comment="#")

    NaNindex = dfin.isnull().any(axis=1).to_numpy().nonzero()[0].tolist()
    dfin.drop(NaNindex, axis=0, inplace=True)

    # 计算 Absorption
    i0 = dfin.loc[:, ["a_i0_1", "a_i0_2"]].sum(axis=1).values
    i1 = dfin.loc[:, ["a_i1_1", "a_i1_2"]].sum(axis=1).values
    dfin["absorption"] = np.log(i0 / i1)  # type: ignore # natural base
    df2out = dfin[columns := ["Pt_No", "tripod_x", "tripod_z", "absorption"]].copy(deep=True)

    # 保存每个能量值下的数据
    df2out.to_csv(Path.joinpath(scan_folder, f"{name_folder}_all.csv"), sep=",", index=False)
    dfout.append(df2out)

    # 新建对应的文件夹以及获取对应的位置
    # 获取切片的引索序列
    scancuttoff_index_list = df2out[(df2out.loc[:, "Pt_No"] == df2out.loc[:, "Pt_No"].min())].index.tolist()
    # scancuttoff_index_list=dfin2[(dfin2.loc[:,'Pt_No']==dfin2.loc[:,'Pt_No'].max())].index.tolist()  # noqa: ERA001
    # print(len(scancuttoff_index_list))  # noqa: ERA001

    # finding global min and max intensity cheapo way for transmission
    vmin = df2out.loc[:, "absorption"].min()
    vmax = df2out.loc[:, "absorption"].max()
    # print(vmin, vmax)  # noqa: ERA001

    for i in trange(len(scancuttoff_index_list) - 1):
        scan_name = f"{name_folder}_{i + 1:04d}"
        path_file_folder = create_folders(scan_folder, name_folder)

        a = scancuttoff_index_list[i]
        b = scancuttoff_index_list[i + 1]

        # 输出每个能量下的每次扫描数据，并画图
        df2out.iloc[a:b, :].to_csv(
            Path.joinpath(path_file_folder, f"{scan_name}.csv"), sep=",", index=True, header=True
        )
        slice_energy = df2out.iloc[a:b, :].pivot(index="tripod_z", columns="tripod_x", values="absorption")
        slice_energy.to_csv(Path.joinpath(path_file_folder, f"{scan_name}_pivot.csv"), sep=",", index=True, header=True)
        figure = data2map(slice_energy, scan_name, todisplay=False, vmin=vmin, vmax=vmax)
        figure.savefig(
            Path.joinpath(path_file_folder, f"{scan_name}.tif"),
            transparent=False,
            pad_inches=0.05,
            bbox_inches="tight",
            dpi=300,
            pil_kwargs={"compression": "tiff_lzw"},
        )
        plt.close()

#### 数据的清洗

In [ ]:
# 裁剪所有 absorption 列到最短长度
min_len = min(len(df) for df in dfout)
Mesh_data = np.array([df.iloc[:min_len, -1].values for df in dfout]).T

# 裁剪前3列位置信息并合并
Mesh_data = pd.concat([dfout[0].iloc[:min_len, :3].reset_index(drop=True), pd.DataFrame(Mesh_data)], axis=1).copy()
Mesh_data.to_csv(
    Path.joinpath(path_out, "Mesh_Absorption_data.csv"),
    sep=",",
    index=False,
    header=[r"Pt_No", r"tripod_x", r"tripod_z", r"absorption1", r"absorption2", r"absorption3", r"absorption4"],  # noqa: E501
)

#### PreEdge 和 PostEdge 的拟合

In [ ]:
from scipy.optimize import minimize

# 数据
E_Zn = np.array([9651.4, 9669.4, 9674.6, 9703.9])
Ra_Zn = np.array([0.01, 0.707741, 2.08340, 0.824950])  # Zn2+
Rb_Zn = np.array([0.01, 0.963532, 1.71398, 0.824950])  # ZHS
Ej_Zn = np.array([0.3860, 1.4200])  # edge jump of Ref. Zn2+, ZHS
constants = [Ra_Zn, Rb_Zn]


# 拟合函数
def model(params, energy, constants):
    eff, deltamu, pre_slope, pre_intercept = params
    Ra, Rb = constants
    return deltamu * (Ra * eff + Rb * (1 - eff)) + pre_slope * energy + pre_intercept


# 目标函数
def objective(params, energy, constants, data):
    pred = model(params, energy, constants)
    mse = np.mean((data - pred) ** 2)
    return mse


# 初始化参数
initial_guess = [0.5, 1.0, 0.0, 0.0]
bounds = [(0, 1), (0, np.inf), (-np.inf, np.inf), (-np.inf, np.inf)]

# 初始化结果容器
params_result = pd.DataFrame(index=Mesh_data.index, columns=["eff", "deltamu", "pre_slope", "pre_intercept"])
absroption_result = pd.DataFrame(index=Mesh_data.index, columns=[f"Normal_{i}" for i in range(Mesh_data.shape[1] - 3)])

# 拟合过程
for i in tqdm(Mesh_data.index):
    temp = Mesh_data.iloc[i, 3:].values
    result = minimize(
        objective, initial_guess, args=(E_Zn, constants, temp), bounds=bounds, method="SLSQP", options={"maxiter": 1000}
    )

    if result.success:
        eff, deltamu, pre_slope, pre_intercept = result.x
        params_result.loc[i] = result.x
        fitted_absorption = deltamu * (Ra_Zn * eff + Rb_Zn * (1 - eff))
        absroption_result.loc[i] = fitted_absorption

#### 验证结果的准确度

In [ ]:
# 保存数据
pd.concat([Mesh_data, absroption_result], axis=1, ignore_index=True).to_csv(
    Path.joinpath(path_out, "Mesh_Absorption_results.csv"),
    sep=",",
    index=False,
    header=[
        r"Pt_No",
        r"tripod_x",
        r"tripod_z",
        r"absorption1",
        r"absorption2",
        r"absorption3",
        r"absorption4",
        r"Normal1",
        r"Normal2",
        r"Normal3",
        r"Normal4",
    ],
)
pd.concat([Mesh_data.iloc[:, :3], params_result], axis=1, ignore_index=True).to_csv(
    Path.joinpath(path_out, "Mesh_Absorption_params.csv"),
    sep=",",
    index=False,
    header=[r"Pt_No", r"tripod_x", r"tripod_z", r"eff", r"deltamu", r"pre_slope", r"pre_intercept"],
)

#### 保存成 tiff 图片

In [ ]:
import tifffile

absroption_result_copy = pd.concat([Mesh_data.iloc[:, :3], absroption_result], axis=1, ignore_index=True)
absroption_result_copy = absroption_result_copy.apply(pd.to_numeric, errors="coerce")


# 每像素 0.5 μm -> 20,000 pixels per cm
resolution = (20000 * 0.32, 10000 * 0.67)  # (x_res, y_res)

scan_folder = create_folders(path_out, "1_Results")
folder_names = [f.parts[-1][:-4] for f in path_master_file]

for idx in trange(3, absroption_result_copy.shape[1]):
    current_folder_name = folder_names[idx - 3]
    path_file_folder = create_folders(scan_folder, current_folder_name)

    columns = absroption_result_copy.columns
    temp = absroption_result_copy.loc[:, [columns[0], columns[1], columns[2], columns[idx]]]
    temp.columns = ["Pt_No", "tripod_x", "tripod_z", "absorption"]

    scancutoff_indices = temp[temp["Pt_No"] == temp["Pt_No"].min()].index.tolist()

    vmin = temp["absorption"].min(skipna=True)
    vmax = temp["absorption"].max(skipna=True)
    fig_list = []
    for i in trange(len(scancutoff_indices) - 1):
        scan_name = f"{current_folder_name}_{i + 1:04d}"
        a, b = scancutoff_indices[i], scancutoff_indices[i + 1]

        slice_df = temp.iloc[a:b, :]
        slice_df.to_csv(Path.joinpath(path_file_folder, f"{scan_name}.csv"), sep=",", index=True, header=True)

        pivot = slice_df.pivot(index="tripod_z", columns="tripod_x", values="absorption")
        pivot.to_csv(Path.joinpath(path_file_folder, f"{scan_name}_pivot.csv"), sep=",", index=True, header=True)
        fig_list.append(pivot.values)
        fig = data2map(pivot, scan_name, todisplay=False, vmin=vmin, vmax=vmax)
        fig.savefig(
            Path.joinpath(path_file_folder, f"{scan_name}.tif"),
            transparent=False,
            pad_inches=0.05,
            bbox_inches="tight",
            dpi=300,
            pil_kwargs={"compression": "tiff_lzw"},
        )
        plt.close()

        tifffile.imwrite(
            Path.joinpath(scan_folder, f"{path_file_folder}.tiff"),
            np.stack(fig_list, axis=0),
            photometric="minisblack",
            compression="lzw",
            resolution=resolution,
            resolutionunit="CENTIMETER",  # or "INCH"
        )

### Zn3

#### 读取数据

In [ ]:
# to read all master files
path_master_file = list(
    Path(
        r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\PaperDos\XAS\Operando\αMnO2\MeshMapping\2023-CLAESS\Data\case2_1stDischarge\DownCell_Mn_2\Zn"  # noqa: E501, RUF001
    ).glob("*.dat")
)
header = "Pt_No tripod_z tripod_x energyc a_i0i1_timer a_i0_1 a_i0_2 a_i1_1 a_i1_2 n_timer n_i0_1 n_i0_2 n_i1_1 n_i1_2 n_i2_1 n_i2_2 n_icr n_tcr n_sca1 n_sca2 n_sca3 n_sca4 sr_i_1 dt"  # noqa: E501
headers = header.split(" ")

In [ ]:
# 处理 mesh 数据
dfout = []
scan_folder = create_folders(path_out, "0_PureAbsorption")
# read data files and clean it
for master_file_name in tqdm(path_master_file):
    name_folder = master_file_name.parts[-1][:-4]

    dfin = pd.read_csv(master_file_name, index_col=None, header=None, names=headers, sep=r"\s+", comment="#")

    NaNindex = dfin.isnull().any(axis=1).to_numpy().nonzero()[0].tolist()
    dfin.drop(NaNindex, axis=0, inplace=True)

    # 计算 Absorption
    i0 = dfin.loc[:, ["a_i0_1", "a_i0_2"]].sum(axis=1).values
    i1 = dfin.loc[:, ["a_i1_1", "a_i1_2"]].sum(axis=1).values
    dfin["absorption"] = np.log(i0 / i1)  # type: ignore # natural base
    df2out = dfin[columns := ["Pt_No", "tripod_x", "tripod_z", "absorption"]].copy(deep=True)

    # 保存每个能量值下的数据
    df2out.to_csv(Path.joinpath(scan_folder, f"{name_folder}_all.csv"), sep=",", index=False)
    dfout.append(df2out)

    # 新建对应的文件夹以及获取对应的位置
    # 获取切片的引索序列
    scancuttoff_index_list = df2out[(df2out.loc[:, "Pt_No"] == df2out.loc[:, "Pt_No"].min())].index.tolist()
    # scancuttoff_index_list=dfin2[(dfin2.loc[:,'Pt_No']==dfin2.loc[:,'Pt_No'].max())].index.tolist()  # noqa: ERA001
    # print(len(scancuttoff_index_list))  # noqa: ERA001

    # finding global min and max intensity cheapo way for transmission
    vmin = df2out.loc[:, "absorption"].min()
    vmax = df2out.loc[:, "absorption"].max()
    # print(vmin, vmax)  # noqa: ERA001

    for i in trange(len(scancuttoff_index_list) - 1):
        scan_name = f"{name_folder}_{i + 1:04d}"
        path_file_folder = create_folders(scan_folder, name_folder)

        a = scancuttoff_index_list[i]
        b = scancuttoff_index_list[i + 1]

        # 输出每个能量下的每次扫描数据，并画图
        df2out.iloc[a:b, :].to_csv(
            Path.joinpath(path_file_folder, f"{scan_name}.csv"), sep=",", index=True, header=True
        )
        slice_energy = df2out.iloc[a:b, :].pivot(index="tripod_z", columns="tripod_x", values="absorption")
        slice_energy.to_csv(Path.joinpath(path_file_folder, f"{scan_name}_pivot.csv"), sep=",", index=True, header=True)
        figure = data2map(slice_energy, scan_name, todisplay=False, vmin=vmin, vmax=vmax)
        figure.savefig(
            Path.joinpath(path_file_folder, f"{scan_name}.tif"),
            transparent=False,
            pad_inches=0.05,
            bbox_inches="tight",
            dpi=300,
            pil_kwargs={"compression": "tiff_lzw"},
        )
        plt.close()

#### 数据的清洗

In [ ]:
# 裁剪所有 absorption 列到最短长度
min_len = min(len(df) for df in dfout)
Mesh_data = np.array([df.iloc[:min_len, -1].values for df in dfout]).T

# 裁剪前3列位置信息并合并
Mesh_data = pd.concat([dfout[0].iloc[:min_len, :3].reset_index(drop=True), pd.DataFrame(Mesh_data)], axis=1).copy()
Mesh_data.to_csv(
    Path.joinpath(path_out, "Mesh_Absorption_data.csv"),
    sep=",",
    index=False,
    header=[r"Pt_No", r"tripod_x", r"tripod_z", r"absorption1", r"absorption2", r"absorption3", r"absorption4"],  # noqa: E501
)

#### PreEdge 和 PostEdge 的拟合

In [ ]:
from scipy.optimize import minimize

# 数据
E_Zn = np.array([9651.4, 9669.4, 9674.6, 9703.9])
Ra_Zn = np.array([0.01, 0.707741, 2.08340, 0.824950])  # Zn2+
Rb_Zn = np.array([0.01, 0.963532, 1.71398, 0.824950])  # ZHS
Ej_Zn = np.array([0.3860, 1.4200])  # edge jump of Ref. Zn2+, ZHS
constants = [Ra_Zn, Rb_Zn]


# 拟合函数
def model(params, energy, constants):
    eff, deltamu, pre_slope, pre_intercept = params
    Ra, Rb = constants
    return deltamu * (Ra * eff + Rb * (1 - eff)) + pre_slope * energy + pre_intercept


# 目标函数
def objective(params, energy, constants, data):
    pred = model(params, energy, constants)
    mse = np.mean((data - pred) ** 2)
    return mse


# 初始化参数
initial_guess = [0.5, 1.0, 0.0, 0.0]
bounds = [(0, 1), (0, np.inf), (-np.inf, np.inf), (-np.inf, np.inf)]

# 初始化结果容器
params_result = pd.DataFrame(index=Mesh_data.index, columns=["eff", "deltamu", "pre_slope", "pre_intercept"])
absroption_result = pd.DataFrame(index=Mesh_data.index, columns=[f"Normal_{i}" for i in range(Mesh_data.shape[1] - 3)])

# 拟合过程
for i in tqdm(Mesh_data.index):
    temp = Mesh_data.iloc[i, 3:].values
    result = minimize(
        objective, initial_guess, args=(E_Zn, constants, temp), bounds=bounds, method="SLSQP", options={"maxiter": 1000}
    )

    if result.success:
        eff, deltamu, pre_slope, pre_intercept = result.x
        params_result.loc[i] = result.x
        fitted_absorption = deltamu * (Ra_Zn * eff + Rb_Zn * (1 - eff))
        absroption_result.loc[i] = fitted_absorption

#### 验证结果的准确度

In [ ]:
# 保存数据
pd.concat([Mesh_data, absroption_result], axis=1, ignore_index=True).to_csv(
    Path.joinpath(path_out, "Mesh_Absorption_results.csv"),
    sep=",",
    index=False,
    header=[
        r"Pt_No",
        r"tripod_x",
        r"tripod_z",
        r"absorption1",
        r"absorption2",
        r"absorption3",
        r"absorption4",
        r"Normal1",
        r"Normal2",
        r"Normal3",
        r"Normal4",
    ],
)
pd.concat([Mesh_data.iloc[:, :3], params_result], axis=1, ignore_index=True).to_csv(
    Path.joinpath(path_out, "Mesh_Absorption_params.csv"),
    sep=",",
    index=False,
    header=[r"Pt_No", r"tripod_x", r"tripod_z", r"eff", r"deltamu", r"pre_slope", r"pre_intercept"],
)

#### 保存成 tiff 图片

In [ ]:
import tifffile

absroption_result_copy = pd.concat([Mesh_data.iloc[:, :3], absroption_result], axis=1, ignore_index=True)
absroption_result_copy = absroption_result_copy.apply(pd.to_numeric, errors="coerce")


# 每像素 0.5 μm -> 20,000 pixels per cm
resolution = (20000 * 0.32, 10000 * 0.67)  # (x_res, y_res)

scan_folder = create_folders(path_out, "1_Results")
folder_names = [f.parts[-1][:-4] for f in path_master_file]

for idx in trange(3, absroption_result_copy.shape[1]):
    current_folder_name = folder_names[idx - 3]
    path_file_folder = create_folders(scan_folder, current_folder_name)

    columns = absroption_result_copy.columns
    temp = absroption_result_copy.loc[:, [columns[0], columns[1], columns[2], columns[idx]]]
    temp.columns = ["Pt_No", "tripod_x", "tripod_z", "absorption"]

    scancutoff_indices = temp[temp["Pt_No"] == temp["Pt_No"].min()].index.tolist()

    vmin = temp["absorption"].min(skipna=True)
    vmax = temp["absorption"].max(skipna=True)
    fig_list = []
    for i in trange(len(scancutoff_indices) - 1):
        scan_name = f"{current_folder_name}_{i + 1:04d}"
        a, b = scancutoff_indices[i], scancutoff_indices[i + 1]

        slice_df = temp.iloc[a:b, :]
        slice_df.to_csv(Path.joinpath(path_file_folder, f"{scan_name}.csv"), sep=",", index=True, header=True)

        pivot = slice_df.pivot(index="tripod_z", columns="tripod_x", values="absorption")
        pivot.to_csv(Path.joinpath(path_file_folder, f"{scan_name}_pivot.csv"), sep=",", index=True, header=True)
        fig_list.append(pivot.values)
        fig = data2map(pivot, scan_name, todisplay=False, vmin=vmin, vmax=vmax)
        fig.savefig(
            Path.joinpath(path_file_folder, f"{scan_name}.tif"),
            transparent=False,
            pad_inches=0.05,
            bbox_inches="tight",
            dpi=300,
            pil_kwargs={"compression": "tiff_lzw"},
        )
        plt.close()

        tifffile.imwrite(
            Path.joinpath(scan_folder, f"{path_file_folder}.tiff"),
            np.stack(fig_list, axis=0),
            photometric="minisblack",
            compression="lzw",
            resolution=resolution,
            resolutionunit="CENTIMETER",  # or "INCH"
        )

### Zn2

#### 读取数据

In [ ]:
# to read all master files
path_master_file = list(
    Path(
        r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\PaperDos\XAS\Operando\αMnO2\MeshMapping\2023-CLAESS\Data\case1_1stCharge_1stDischarge\UpCell_MnFree\Zn"  # noqa: E501, RUF001
    ).glob("*.dat")
)
header = "Pt_No tripod_z tripod_x energyc a_i0i1_timer a_i0_1 a_i0_2 a_i1_1 a_i1_2 n_timer n_i0_1 n_i0_2 n_i1_1 n_i1_2 n_i2_1 n_i2_2 n_icr n_tcr n_sca1 n_sca2 n_sca3 n_sca4 sr_i_1 dt"  # noqa: E501
headers = header.split(" ")

In [ ]:
# 处理 mesh 数据
dfout = []
scan_folder = create_folders(path_out, "0_PureAbsorption")
# read data files and clean it
for master_file_name in tqdm(path_master_file):
    name_folder = master_file_name.parts[-1][:-4]

    dfin = pd.read_csv(master_file_name, index_col=None, header=None, names=headers, sep=r"\s+", comment="#")

    NaNindex = dfin.isnull().any(axis=1).to_numpy().nonzero()[0].tolist()
    dfin.drop(NaNindex, axis=0, inplace=True)

    # 计算 Absorption
    i0 = dfin.loc[:, ["a_i0_1", "a_i0_2"]].sum(axis=1).values
    i1 = dfin.loc[:, ["a_i1_1", "a_i1_2"]].sum(axis=1).values
    dfin["absorption"] = np.log(i0 / i1)  # type: ignore # natural base
    df2out = dfin[columns := ["Pt_No", "tripod_x", "tripod_z", "absorption"]].copy(deep=True)

    # 保存每个能量值下的数据
    df2out.to_csv(Path.joinpath(scan_folder, f"{name_folder}_all.csv"), sep=",", index=False)
    dfout.append(df2out)

    # 新建对应的文件夹以及获取对应的位置
    # 获取切片的引索序列
    scancuttoff_index_list = df2out[(df2out.loc[:, "Pt_No"] == df2out.loc[:, "Pt_No"].min())].index.tolist()
    # scancuttoff_index_list=dfin2[(dfin2.loc[:,'Pt_No']==dfin2.loc[:,'Pt_No'].max())].index.tolist()  # noqa: ERA001
    # print(len(scancuttoff_index_list))  # noqa: ERA001

    # finding global min and max intensity cheapo way for transmission
    vmin = df2out.loc[:, "absorption"].min()
    vmax = df2out.loc[:, "absorption"].max()
    # print(vmin, vmax)  # noqa: ERA001

    for i in trange(len(scancuttoff_index_list) - 1):
        scan_name = f"{name_folder}_{i + 1:04d}"
        path_file_folder = create_folders(scan_folder, name_folder)

        a = scancuttoff_index_list[i]
        b = scancuttoff_index_list[i + 1]

        # 输出每个能量下的每次扫描数据，并画图
        df2out.iloc[a:b, :].to_csv(
            Path.joinpath(path_file_folder, f"{scan_name}.csv"), sep=",", index=True, header=True
        )
        slice_energy = df2out.iloc[a:b, :].pivot(index="tripod_z", columns="tripod_x", values="absorption")
        slice_energy.to_csv(Path.joinpath(path_file_folder, f"{scan_name}_pivot.csv"), sep=",", index=True, header=True)
        figure = data2map(slice_energy, scan_name, todisplay=False, vmin=vmin, vmax=vmax)
        figure.savefig(
            Path.joinpath(path_file_folder, f"{scan_name}.tif"),
            transparent=False,
            pad_inches=0.05,
            bbox_inches="tight",
            dpi=300,
            pil_kwargs={"compression": "tiff_lzw"},
        )
        plt.close()

#### 数据的清洗

In [ ]:
# 裁剪所有 absorption 列到最短长度
min_len = min(len(df) for df in dfout)
Mesh_data = np.array([df.iloc[:min_len, -1].values for df in dfout]).T

# 裁剪前3列位置信息并合并
Mesh_data = pd.concat([dfout[0].iloc[:min_len, :3].reset_index(drop=True), pd.DataFrame(Mesh_data)], axis=1).copy()
Mesh_data.to_csv(
    Path.joinpath(path_out, "Mesh_Absorption_data.csv"),
    sep=",",
    index=False,
    header=[r"Pt_No", r"tripod_x", r"tripod_z", r"absorption1", r"absorption2", r"absorption3", r"absorption4"],  # noqa: E501
)

#### PreEdge 和 PostEdge 的拟合

In [ ]:
from scipy.optimize import minimize

# 数据
E_Zn = np.array([9651.4, 9669.4, 9674.6, 9703.9])
Ra_Zn = np.array([0.01, 0.707741, 2.08340, 0.824950])  # Zn2+
Rb_Zn = np.array([0.01, 0.963532, 1.71398, 0.824950])  # ZHS
Ej_Zn = np.array([0.3860, 1.4200])  # edge jump of Ref. Zn2+, ZHS
constants = [Ra_Zn, Rb_Zn]


# 拟合函数
def model(params, energy, constants):
    eff, deltamu, pre_slope, pre_intercept = params
    Ra, Rb = constants
    return deltamu * (Ra * eff + Rb * (1 - eff)) + pre_slope * energy + pre_intercept


# 目标函数
def objective(params, energy, constants, data):
    pred = model(params, energy, constants)
    mse = np.mean((data - pred) ** 2)
    return mse


# 初始化参数
initial_guess = [0.5, 1.0, 0.0, 0.0]
bounds = [(0, 1), (0, np.inf), (-np.inf, np.inf), (-np.inf, np.inf)]

# 初始化结果容器
params_result = pd.DataFrame(index=Mesh_data.index, columns=["eff", "deltamu", "pre_slope", "pre_intercept"])
absroption_result = pd.DataFrame(index=Mesh_data.index, columns=[f"Normal_{i}" for i in range(Mesh_data.shape[1] - 3)])

# 拟合过程
for i in tqdm(Mesh_data.index):
    temp = Mesh_data.iloc[i, 3:].values
    result = minimize(
        objective, initial_guess, args=(E_Zn, constants, temp), bounds=bounds, method="SLSQP", options={"maxiter": 1000}
    )

    if result.success:
        eff, deltamu, pre_slope, pre_intercept = result.x
        params_result.loc[i] = result.x
        fitted_absorption = deltamu * (Ra_Zn * eff + Rb_Zn * (1 - eff))
        absroption_result.loc[i] = fitted_absorption

#### 验证结果的准确度

In [ ]:
# 保存数据
pd.concat([Mesh_data, absroption_result], axis=1, ignore_index=True).to_csv(
    Path.joinpath(path_out, "Mesh_Absorption_results.csv"),
    sep=",",
    index=False,
    header=[
        r"Pt_No",
        r"tripod_x",
        r"tripod_z",
        r"absorption1",
        r"absorption2",
        r"absorption3",
        r"absorption4",
        r"Normal1",
        r"Normal2",
        r"Normal3",
        r"Normal4",
    ],
)
pd.concat([Mesh_data.iloc[:, :3], params_result], axis=1, ignore_index=True).to_csv(
    Path.joinpath(path_out, "Mesh_Absorption_params.csv"),
    sep=",",
    index=False,
    header=[r"Pt_No", r"tripod_x", r"tripod_z", r"eff", r"deltamu", r"pre_slope", r"pre_intercept"],
)

#### 保存成 tiff 图片

In [ ]:
import tifffile

absroption_result_copy = pd.concat([Mesh_data.iloc[:, :3], absroption_result], axis=1, ignore_index=True)
absroption_result_copy = absroption_result_copy.apply(pd.to_numeric, errors="coerce")


# 每像素 0.5 μm -> 20,000 pixels per cm
resolution = (20000 * 0.32, 10000 * 0.67)  # (x_res, y_res)

scan_folder = create_folders(path_out, "1_Results")
folder_names = [f.parts[-1][:-4] for f in path_master_file]

for idx in trange(3, absroption_result_copy.shape[1]):
    current_folder_name = folder_names[idx - 3]
    path_file_folder = create_folders(scan_folder, current_folder_name)

    columns = absroption_result_copy.columns
    temp = absroption_result_copy.loc[:, [columns[0], columns[1], columns[2], columns[idx]]]
    temp.columns = ["Pt_No", "tripod_x", "tripod_z", "absorption"]

    scancutoff_indices = temp[temp["Pt_No"] == temp["Pt_No"].min()].index.tolist()

    vmin = temp["absorption"].min(skipna=True)
    vmax = temp["absorption"].max(skipna=True)
    fig_list = []
    for i in trange(len(scancutoff_indices) - 1):
        scan_name = f"{current_folder_name}_{i + 1:04d}"
        a, b = scancutoff_indices[i], scancutoff_indices[i + 1]

        slice_df = temp.iloc[a:b, :]
        slice_df.to_csv(Path.joinpath(path_file_folder, f"{scan_name}.csv"), sep=",", index=True, header=True)

        pivot = slice_df.pivot(index="tripod_z", columns="tripod_x", values="absorption")
        pivot.to_csv(Path.joinpath(path_file_folder, f"{scan_name}_pivot.csv"), sep=",", index=True, header=True)
        fig_list.append(pivot.values)
        fig = data2map(pivot, scan_name, todisplay=False, vmin=vmin, vmax=vmax)
        fig.savefig(
            Path.joinpath(path_file_folder, f"{scan_name}.tif"),
            transparent=False,
            pad_inches=0.05,
            bbox_inches="tight",
            dpi=300,
            pil_kwargs={"compression": "tiff_lzw"},
        )
        plt.close()

        tifffile.imwrite(
            Path.joinpath(scan_folder, f"{path_file_folder}.tiff"),
            np.stack(fig_list, axis=0),
            photometric="minisblack",
            compression="lzw",
            resolution=resolution,
            resolutionunit="CENTIMETER",  # or "INCH"
        )

### Zn1

#### 读取数据

In [ ]:
# to read all master files
path_master_file = list(
    Path(
        r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\PaperDos\XAS\Operando\αMnO2\MeshMapping\2023-CLAESS\Data\case1_1stCharge_1stDischarge\DownCell_Mn\Zn"  # noqa: E501, RUF001
    ).glob("*.dat")
)
header = "Pt_No tripod_z tripod_x energyc a_i0i1_timer a_i0_1 a_i0_2 a_i1_1 a_i1_2 n_timer n_i0_1 n_i0_2 n_i1_1 n_i1_2 n_i2_1 n_i2_2 n_icr n_tcr n_sca1 n_sca2 n_sca3 n_sca4 sr_i_1 dt"  # noqa: E501
headers = header.split(" ")

In [ ]:
# 处理 mesh 数据
dfout = []
scan_folder = create_folders(path_out, "0_PureAbsorption")
# read data files and clean it
for master_file_name in tqdm(path_master_file):
    name_folder = master_file_name.parts[-1][:-4]

    dfin = pd.read_csv(master_file_name, index_col=None, header=None, names=headers, sep=r"\s+", comment="#")

    NaNindex = dfin.isnull().any(axis=1).to_numpy().nonzero()[0].tolist()
    dfin.drop(NaNindex, axis=0, inplace=True)

    # 计算 Absorption
    i0 = dfin.loc[:, ["a_i0_1", "a_i0_2"]].sum(axis=1).values
    i1 = dfin.loc[:, ["a_i1_1", "a_i1_2"]].sum(axis=1).values
    dfin["absorption"] = np.log(i0 / i1)  # type: ignore # natural base
    df2out = dfin[columns := ["Pt_No", "tripod_x", "tripod_z", "absorption"]].copy(deep=True)

    # 保存每个能量值下的数据
    df2out.to_csv(Path.joinpath(scan_folder, f"{name_folder}_all.csv"), sep=",", index=False)
    dfout.append(df2out)

    # 新建对应的文件夹以及获取对应的位置
    # 获取切片的引索序列
    scancuttoff_index_list = df2out[(df2out.loc[:, "Pt_No"] == df2out.loc[:, "Pt_No"].min())].index.tolist()
    # scancuttoff_index_list=dfin2[(dfin2.loc[:,'Pt_No']==dfin2.loc[:,'Pt_No'].max())].index.tolist()  # noqa: ERA001
    # print(len(scancuttoff_index_list))  # noqa: ERA001

    # finding global min and max intensity cheapo way for transmission
    vmin = df2out.loc[:, "absorption"].min()
    vmax = df2out.loc[:, "absorption"].max()
    # print(vmin, vmax)  # noqa: ERA001

    for i in trange(len(scancuttoff_index_list) - 1):
        scan_name = f"{name_folder}_{i + 1:04d}"
        path_file_folder = create_folders(scan_folder, name_folder)

        a = scancuttoff_index_list[i]
        b = scancuttoff_index_list[i + 1]

        # 输出每个能量下的每次扫描数据，并画图
        df2out.iloc[a:b, :].to_csv(
            Path.joinpath(path_file_folder, f"{scan_name}.csv"), sep=",", index=True, header=True
        )
        slice_energy = df2out.iloc[a:b, :].pivot(index="tripod_z", columns="tripod_x", values="absorption")
        slice_energy.to_csv(Path.joinpath(path_file_folder, f"{scan_name}_pivot.csv"), sep=",", index=True, header=True)
        figure = data2map(slice_energy, scan_name, todisplay=False, vmin=vmin, vmax=vmax)
        figure.savefig(
            Path.joinpath(path_file_folder, f"{scan_name}.tif"),
            transparent=False,
            pad_inches=0.05,
            bbox_inches="tight",
            dpi=300,
            pil_kwargs={"compression": "tiff_lzw"},
        )
        plt.close()

#### 数据的清洗

In [ ]:
# 裁剪所有 absorption 列到最短长度
min_len = min(len(df) for df in dfout)
Mesh_data = np.array([df.iloc[:min_len, -1].values for df in dfout]).T

# 裁剪前3列位置信息并合并
Mesh_data = pd.concat([dfout[0].iloc[:min_len, :3].reset_index(drop=True), pd.DataFrame(Mesh_data)], axis=1).copy()
Mesh_data.to_csv(
    Path.joinpath(path_out, "Mesh_Absorption_data.csv"),
    sep=",",
    index=False,
    header=[r"Pt_No", r"tripod_x", r"tripod_z", r"absorption1", r"absorption2", r"absorption3", r"absorption4"],  # noqa: E501
)

#### PreEdge 和 PostEdge 的拟合

In [ ]:
from scipy.optimize import minimize

# 数据
E_Zn = np.array([9651.4, 9669.4, 9674.6, 9703.9])
Ra_Zn = np.array([0.01, 0.707741, 2.08340, 0.824950])  # Zn2+
Rb_Zn = np.array([0.01, 0.963532, 1.71398, 0.824950])  # ZHS
Ej_Zn = np.array([0.3860, 1.4200])  # edge jump of Ref. Zn2+, ZHS
constants = [Ra_Zn, Rb_Zn]


# 拟合函数
def model(params, energy, constants):
    eff, deltamu, pre_slope, pre_intercept = params
    Ra, Rb = constants
    return deltamu * (Ra * eff + Rb * (1 - eff)) + pre_slope * energy + pre_intercept


# 目标函数
def objective(params, energy, constants, data):
    pred = model(params, energy, constants)
    mse = np.mean((data - pred) ** 2)
    return mse


# 初始化参数
initial_guess = [0.5, 1.0, 0.0, 0.0]
bounds = [(0, 1), (0, np.inf), (-np.inf, np.inf), (-np.inf, np.inf)]

# 初始化结果容器
params_result = pd.DataFrame(index=Mesh_data.index, columns=["eff", "deltamu", "pre_slope", "pre_intercept"])
absroption_result = pd.DataFrame(index=Mesh_data.index, columns=[f"Normal_{i}" for i in range(Mesh_data.shape[1] - 3)])
Zn2 = pd.DataFrame(index=Mesh_data.index, columns=[f"Normal_{i}" for i in range(Mesh_data.shape[1] - 3)])
ZHS = pd.DataFrame(index=Mesh_data.index, columns=[f"Normal_{i}" for i in range(Mesh_data.shape[1] - 3)])

# 拟合过程
for i in tqdm(Mesh_data.index):
    temp = Mesh_data.iloc[i, 3:].values
    result = minimize(
        objective, initial_guess, args=(E_Zn, constants, temp), bounds=bounds, method="SLSQP", options={"maxiter": 1000}
    )

    if result.success:
        eff, deltamu, pre_slope, pre_intercept = result.x
        params_result.loc[i] = result.x
        fitted_absorption = deltamu * (Ra_Zn * eff + Rb_Zn * (1 - eff))
        absroption_result.loc[i] = fitted_absorption
        Zn2.loc[i] = deltamu * Ra_Zn * eff
        ZHS.loc[i] = deltamu * Rb_Zn * (1 - eff)

#### 验证结果的准确度

In [ ]:
# 保存数据
pd.concat([Mesh_data, absroption_result], axis=1, ignore_index=True).to_csv(
    Path.joinpath(path_out, "Mesh_Absorption_results.csv"),
    sep=",",
    index=False,
    header=[
        r"Pt_No",
        r"tripod_x",
        r"tripod_z",
        r"absorption1",
        r"absorption2",
        r"absorption3",
        r"absorption4",
        r"Normal1",
        r"Normal2",
        r"Normal3",
        r"Normal4",
    ],
)
pd.concat([Mesh_data.iloc[:, :3], params_result], axis=1, ignore_index=True).to_csv(
    Path.joinpath(path_out, "Mesh_Absorption_params.csv"),
    sep=",",
    index=False,
    header=[r"Pt_No", r"tripod_x", r"tripod_z", r"eff", r"deltamu", r"pre_slope", r"pre_intercept"],
)

In [ ]:
# 保存数据
pd.concat([Mesh_data, Zn2], axis=1, ignore_index=True).to_csv(
    Path.joinpath(path_out, "Mesh_Absorption_results_Zn2+.csv"),
    sep=",",
    index=False,
    header=[
        r"Pt_No",
        r"tripod_x",
        r"tripod_z",
        r"absorption1",
        r"absorption2",
        r"absorption3",
        r"absorption4",
        r"Zn2_1",
        r"Zn2_2",
        r"Zn2_3",
        r"Zn2_4",
    ],
)
pd.concat([Mesh_data, ZHS], axis=1, ignore_index=True).to_csv(
    Path.joinpath(path_out, "Mesh_Absorption_results_Zn2+.csv"),
    sep=",",
    index=False,
    header=[
        r"Pt_No",
        r"tripod_x",
        r"tripod_z",
        r"absorption1",
        r"absorption2",
        r"absorption3",
        r"absorption4",
        r"ZHS_1",
        r"ZHS_2",
        r"ZHS_3",
        r"ZHS_4",
    ],
)

#### 保存成 tiff 图片

In [ ]:
import tifffile

absroption_result_copy = pd.concat([Mesh_data.iloc[:, :3], absroption_result], axis=1, ignore_index=True)
absroption_result_copy = absroption_result_copy.apply(pd.to_numeric, errors="coerce")


# 每像素 0.5 μm -> 20,000 pixels per cm
resolution = (20000 * 0.32, 10000 * 0.67)  # (x_res, y_res)

scan_folder = create_folders(path_out, "1_Results")
folder_names = [f.parts[-1][:-4] for f in path_master_file]

for idx in trange(3, absroption_result_copy.shape[1]):
    current_folder_name = folder_names[idx - 3]
    path_file_folder = create_folders(scan_folder, current_folder_name)

    columns = absroption_result_copy.columns
    temp = absroption_result_copy.loc[:, [columns[0], columns[1], columns[2], columns[idx]]]
    temp.columns = ["Pt_No", "tripod_x", "tripod_z", "absorption"]

    scancutoff_indices = temp[temp["Pt_No"] == temp["Pt_No"].min()].index.tolist()

    vmin = temp["absorption"].min(skipna=True)
    vmax = temp["absorption"].max(skipna=True)
    fig_list = []
    for i in trange(len(scancutoff_indices) - 1):
        scan_name = f"{current_folder_name}_{i + 1:04d}"
        a, b = scancutoff_indices[i], scancutoff_indices[i + 1]

        slice_df = temp.iloc[a:b, :]
        slice_df.to_csv(Path.joinpath(path_file_folder, f"{scan_name}.csv"), sep=",", index=True, header=True)

        pivot = slice_df.pivot(index="tripod_z", columns="tripod_x", values="absorption")
        pivot.to_csv(Path.joinpath(path_file_folder, f"{scan_name}_pivot.csv"), sep=",", index=True, header=True)
        fig_list.append(pivot.values)
        fig = data2map(pivot, scan_name, todisplay=False, vmin=vmin, vmax=vmax)
        fig.savefig(
            Path.joinpath(path_file_folder, f"{scan_name}.tif"),
            transparent=False,
            pad_inches=0.05,
            bbox_inches="tight",
            dpi=300,
            pil_kwargs={"compression": "tiff_lzw"},
        )
        plt.close()

        tifffile.imwrite(
            Path.joinpath(scan_folder, f"{path_file_folder}.tiff"),
            np.stack(fig_list, axis=0),
            photometric="minisblack",
            compression="lzw",
            resolution=resolution,
            resolutionunit="CENTIMETER",  # or "INCH"
        )

In [ ]:
import tifffile

absroption_result_copy = pd.concat([Mesh_data.iloc[:, :3], Zn2], axis=1, ignore_index=True)
absroption_result_copy = absroption_result_copy.apply(pd.to_numeric, errors="coerce")


# 每像素 0.5 μm -> 20,000 pixels per cm
resolution = (20000 * 0.32, 10000 * 0.67)  # (x_res, y_res)

scan_folder = create_folders(path_out, "1_Results_Zn2")
folder_names = [f.parts[-1][:-4] for f in path_master_file]

for idx in trange(3, absroption_result_copy.shape[1]):
    current_folder_name = folder_names[idx - 3]
    path_file_folder = create_folders(scan_folder, current_folder_name)

    columns = absroption_result_copy.columns
    temp = absroption_result_copy.loc[:, [columns[0], columns[1], columns[2], columns[idx]]]
    temp.columns = ["Pt_No", "tripod_x", "tripod_z", "absorption"]

    scancutoff_indices = temp[temp["Pt_No"] == temp["Pt_No"].min()].index.tolist()

    vmin = temp["absorption"].min(skipna=True)
    vmax = temp["absorption"].max(skipna=True)
    fig_list = []
    for i in trange(len(scancutoff_indices) - 1):
        scan_name = f"{current_folder_name}_{i + 1:04d}"
        a, b = scancutoff_indices[i], scancutoff_indices[i + 1]

        slice_df = temp.iloc[a:b, :]
        slice_df.to_csv(Path.joinpath(path_file_folder, f"{scan_name}.csv"), sep=",", index=True, header=True)

        pivot = slice_df.pivot(index="tripod_z", columns="tripod_x", values="absorption")
        pivot.to_csv(Path.joinpath(path_file_folder, f"{scan_name}_pivot.csv"), sep=",", index=True, header=True)
        fig_list.append(pivot.values)
        fig = data2map(pivot, scan_name, todisplay=False, vmin=vmin, vmax=vmax)
        fig.savefig(
            Path.joinpath(path_file_folder, f"{scan_name}.tif"),
            transparent=False,
            pad_inches=0.05,
            bbox_inches="tight",
            dpi=300,
            pil_kwargs={"compression": "tiff_lzw"},
        )
        plt.close()

        tifffile.imwrite(
            Path.joinpath(scan_folder, f"{path_file_folder}.tiff"),
            np.stack(fig_list, axis=0),
            photometric="minisblack",
            compression="lzw",
            resolution=resolution,
            resolutionunit="CENTIMETER",  # or "INCH"
        )

In [ ]:
import tifffile

absroption_result_copy = pd.concat([Mesh_data.iloc[:, :3], ZHS], axis=1, ignore_index=True)
absroption_result_copy = absroption_result_copy.apply(pd.to_numeric, errors="coerce")


# 每像素 0.5 μm -> 20,000 pixels per cm
resolution = (20000 * 0.32, 10000 * 0.67)  # (x_res, y_res)

scan_folder = create_folders(path_out, "1_Results_ZHS")
folder_names = [f.parts[-1][:-4] for f in path_master_file]

for idx in trange(3, absroption_result_copy.shape[1]):
    current_folder_name = folder_names[idx - 3]
    path_file_folder = create_folders(scan_folder, current_folder_name)

    columns = absroption_result_copy.columns
    temp = absroption_result_copy.loc[:, [columns[0], columns[1], columns[2], columns[idx]]]
    temp.columns = ["Pt_No", "tripod_x", "tripod_z", "absorption"]

    scancutoff_indices = temp[temp["Pt_No"] == temp["Pt_No"].min()].index.tolist()

    vmin = temp["absorption"].min(skipna=True)
    vmax = temp["absorption"].max(skipna=True)
    fig_list = []
    for i in trange(len(scancutoff_indices) - 1):
        scan_name = f"{current_folder_name}_{i + 1:04d}"
        a, b = scancutoff_indices[i], scancutoff_indices[i + 1]

        slice_df = temp.iloc[a:b, :]
        slice_df.to_csv(Path.joinpath(path_file_folder, f"{scan_name}.csv"), sep=",", index=True, header=True)

        pivot = slice_df.pivot(index="tripod_z", columns="tripod_x", values="absorption")
        pivot.to_csv(Path.joinpath(path_file_folder, f"{scan_name}_pivot.csv"), sep=",", index=True, header=True)
        fig_list.append(pivot.values)
        fig = data2map(pivot, scan_name, todisplay=False, vmin=vmin, vmax=vmax)
        fig.savefig(
            Path.joinpath(path_file_folder, f"{scan_name}.tif"),
            transparent=False,
            pad_inches=0.05,
            bbox_inches="tight",
            dpi=300,
            pil_kwargs={"compression": "tiff_lzw"},
        )
        plt.close()

        tifffile.imwrite(
            Path.joinpath(scan_folder, f"{path_file_folder}.tiff"),
            np.stack(fig_list, axis=0),
            photometric="minisblack",
            compression="lzw",
            resolution=resolution,
            resolutionunit="CENTIMETER",  # or "INCH"
        )

### Mn4

#### 读取数据

In [ ]:
# to read all master files
path_master_file = list(
    Path(
        r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\PaperDos\XAS\Operando\αMnO2\MeshMapping\2023-CLAESS\Data\case2_1stDischarge\DownCell_Mn_2\Mn"  # noqa: E501, RUF001
    ).glob("*.dat")
)
header = "Pt_No tripod_z tripod_x energyc a_i0i1_timer a_i0_1 a_i0_2 a_i1_1 a_i1_2 n_timer n_i0_1 n_i0_2 n_i1_1 n_i1_2 n_i2_1 n_i2_2 n_icr n_tcr n_sca1 n_sca2 n_sca3 n_sca4 sr_i_1 dt"  # noqa: E501
headers = header.split(" ")

In [ ]:
# 处理 mesh 数据
dfout = []
scan_folder = create_folders(path_out, "0_PureAbsorption")
# read data files and clean it
for master_file_name in tqdm(path_master_file):
    name_folder = master_file_name.parts[-1][:-4]

    dfin = pd.read_csv(master_file_name, index_col=None, header=None, names=headers, sep=r"\s+", comment="#")

    NaNindex = dfin.isnull().any(axis=1).to_numpy().nonzero()[0].tolist()
    dfin.drop(NaNindex, axis=0, inplace=True)

    # 计算 Absorption
    i0 = dfin.loc[:, ["a_i0_1", "a_i0_2"]].sum(axis=1).values
    i1 = dfin.loc[:, ["a_i1_1", "a_i1_2"]].sum(axis=1).values
    dfin["absorption"] = np.log(i0 / i1)  # type: ignore # natural base
    df2out = dfin[columns := ["Pt_No", "tripod_x", "tripod_z", "absorption"]].copy(deep=True)

    # 保存每个能量值下的数据
    df2out.to_csv(Path.joinpath(scan_folder, f"{name_folder}_all.csv"), sep=",", index=False)
    dfout.append(df2out)

    # 新建对应的文件夹以及获取对应的位置
    # 获取切片的引索序列
    scancuttoff_index_list = df2out[(df2out.loc[:, "Pt_No"] == df2out.loc[:, "Pt_No"].min())].index.tolist()
    # scancuttoff_index_list=dfin2[(dfin2.loc[:,'Pt_No']==dfin2.loc[:,'Pt_No'].max())].index.tolist()  # noqa: ERA001
    # print(len(scancuttoff_index_list))  # noqa: ERA001

    # finding global min and max intensity cheapo way for transmission
    vmin = df2out.loc[:, "absorption"].min()
    vmax = df2out.loc[:, "absorption"].max()
    # print(vmin, vmax)  # noqa: ERA001

    for i in trange(len(scancuttoff_index_list) - 1):
        scan_name = f"{name_folder}_{i + 1:04d}"
        path_file_folder = create_folders(scan_folder, name_folder)

        a = scancuttoff_index_list[i]
        b = scancuttoff_index_list[i + 1]

        # 输出每个能量下的每次扫描数据，并画图
        df2out.iloc[a:b, :].to_csv(
            Path.joinpath(path_file_folder, f"{scan_name}.csv"), sep=",", index=True, header=True
        )
        slice_energy = df2out.iloc[a:b, :].pivot(index="tripod_z", columns="tripod_x", values="absorption")
        slice_energy.to_csv(Path.joinpath(path_file_folder, f"{scan_name}_pivot.csv"), sep=",", index=True, header=True)
        figure = data2map(slice_energy, scan_name, todisplay=False, vmin=vmin, vmax=vmax)
        figure.savefig(
            Path.joinpath(path_file_folder, f"{scan_name}.tif"),
            transparent=False,
            pad_inches=0.05,
            bbox_inches="tight",
            dpi=300,
            pil_kwargs={"compression": "tiff_lzw"},
        )
        plt.close()

#### 数据的清洗

In [ ]:
# 裁剪所有 absorption 列到最短长度
min_len = min(len(df) for df in dfout)
Mesh_data = np.array([df.iloc[:min_len, -1].values for df in dfout]).T

# 裁剪前3列位置信息并合并
Mesh_data = pd.concat([dfout[0].iloc[:min_len, :3].reset_index(drop=True), pd.DataFrame(Mesh_data)], axis=1).copy()
Mesh_data.to_csv(
    Path.joinpath(path_out, "Mesh_Absorption_data.csv"),
    sep=",",
    index=False,
    header=[
        r"Pt_No",
        r"tripod_x",
        r"tripod_z",
        r"absorption1",
        r"absorption2",
        r"absorption3",
        r"absorption4",
        r"absorption5",
    ],  # noqa: E501
)

#### PreEdge 和 PostEdge 的拟合

In [ ]:
from scipy.optimize import minimize

# 数据
E_Mn = np.array([6533.0, 6553.2, 6557.5, 6561.5, 6610.8])
Ra_Mn = np.array([0.01, 1.95670, 1.19821, 1.03919, 1.04312])
Rb_Mn = np.array([0.01, 0.531024, 1.04128, 1.42149, 0.983545])
constants = [Ra_Mn, Rb_Mn]


# 拟合函数
def model(params, energy, constants):
    eff1, eff2, deltamu, pre_slope, pre_intercept = params
    Ra, Rb = constants
    return deltamu * (Ra * eff1 + Rb * eff2) + pre_slope * energy + pre_intercept


# 目标函数
def objective(params, energy, constants, data):
    pred = model(params, energy, constants)
    mse = np.mean((data - pred) ** 2)
    eff1, eff2 = params[0], params[1]
    penalty = 100 * (eff1 + eff2 - 1) ** 2
    return mse + penalty


# 初始化参数
initial_guess = [0.5, 0.5, 1.0, 0.0, 0.0]
bounds = [(0, 1), (0, 1), (0, np.inf), (-np.inf, np.inf), (-np.inf, np.inf)]

# 初始化结果容器
params_result = pd.DataFrame(index=Mesh_data.index, columns=["eff1", "eff2", "deltamu", "pre_slope", "pre_intercept"])
absroption_result = pd.DataFrame(index=Mesh_data.index, columns=[f"Normal_{i}" for i in range(Mesh_data.shape[1] - 3)])

# 拟合过程
for i in tqdm(Mesh_data.index):
    temp = Mesh_data.iloc[i, 3:].values
    result = minimize(
        objective, initial_guess, args=(E_Mn, constants, temp), bounds=bounds, method="SLSQP", options={"maxiter": 1000}
    )

    if result.success:
        eff1, eff2, deltamu, pre_slope, pre_intercept = result.x
        params_result.loc[i] = result.x
        fitted_absorption = deltamu * (Ra_Mn * eff1 + Rb_Mn * eff2)
        absroption_result.loc[i] = fitted_absorption[:5]

#### 验证结果的准确度

In [ ]:
# 保存数据
pd.concat([Mesh_data, absroption_result], axis=1, ignore_index=True).to_csv(
    Path.joinpath(path_out, "Mesh_Absorption_results.csv"),
    sep=",",
    index=False,
    header=[
        r"Pt_No",
        r"tripod_x",
        r"tripod_z",
        r"absorption1",
        r"absorption2",
        r"absorption3",
        r"absorption4",
        r"absorption5",
        r"Normal1",
        r"Normal2",
        r"Normal3",
        r"Normal4",
        r"Normal5",
    ],
)
pd.concat([Mesh_data.iloc[:, :3], params_result], axis=1, ignore_index=True).to_csv(
    Path.joinpath(path_out, "Mesh_Absorption_params.csv"),
    sep=",",
    index=False,
    header=[r"Pt_No", r"tripod_x", r"tripod_z", r"eff1", r"eff2", r"deltamu", r"pre_slope", r"pre_intercept"],
)

#### 保存成 tiff 图片

In [ ]:
import tifffile

absroption_result_copy = pd.concat([Mesh_data.iloc[:, :3], absroption_result], axis=1, ignore_index=True)
absroption_result_copy = absroption_result_copy.apply(pd.to_numeric, errors="coerce")


# 每像素 0.5 μm -> 20,000 pixels per cm
resolution = (20000 * 0.32, 10000 * 0.67)  # (x_res, y_res)

scan_folder = create_folders(path_out, "1_Results")
folder_names = [f.parts[-1][:-4] for f in path_master_file]

for idx in trange(3, absroption_result_copy.shape[1]):
    current_folder_name = folder_names[idx - 3]
    path_file_folder = create_folders(scan_folder, current_folder_name)

    columns = absroption_result_copy.columns
    temp = absroption_result_copy.loc[:, [columns[0], columns[1], columns[2], columns[idx]]]
    temp.columns = ["Pt_No", "tripod_x", "tripod_z", "absorption"]

    scancutoff_indices = temp[temp["Pt_No"] == temp["Pt_No"].min()].index.tolist()

    vmin = temp["absorption"].min(skipna=True)
    vmax = temp["absorption"].max(skipna=True)
    fig_list = []
    for i in trange(len(scancutoff_indices) - 1):
        scan_name = f"{current_folder_name}_{i + 1:04d}"
        a, b = scancutoff_indices[i], scancutoff_indices[i + 1]

        slice_df = temp.iloc[a:b, :]
        slice_df.to_csv(Path.joinpath(path_file_folder, f"{scan_name}.csv"), sep=",", index=True, header=True)

        pivot = slice_df.pivot(index="tripod_z", columns="tripod_x", values="absorption")
        pivot.to_csv(Path.joinpath(path_file_folder, f"{scan_name}_pivot.csv"), sep=",", index=True, header=True)
        fig_list.append(pivot.values)
        fig = data2map(pivot, scan_name, todisplay=False, vmin=vmin, vmax=vmax)
        fig.savefig(
            Path.joinpath(path_file_folder, f"{scan_name}.tif"),
            transparent=False,
            pad_inches=0.05,
            bbox_inches="tight",
            dpi=300,
            pil_kwargs={"compression": "tiff_lzw"},
        )
        plt.close()

        tifffile.imwrite(
            Path.joinpath(scan_folder, f"{path_file_folder}.tiff"),
            np.stack(fig_list, axis=0),
            photometric="minisblack",
            compression="lzw",
            resolution=resolution,
            resolutionunit="CENTIMETER",  # or "INCH"
        )

### Mn3

#### 读取数据

In [ ]:
# to read all master files
path_master_file = list(
    Path(
        r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\PaperDos\XAS\Operando\αMnO2\MeshMapping\2023-CLAESS\Data\case2_1stDischarge\UpCell_MnFree_2\Mn"  # noqa: E501, RUF001
    ).glob("*.dat")
)
header = "Pt_No tripod_z tripod_x energyc a_i0i1_timer a_i0_1 a_i0_2 a_i1_1 a_i1_2 n_timer n_i0_1 n_i0_2 n_i1_1 n_i1_2 n_i2_1 n_i2_2 n_icr n_tcr n_sca1 n_sca2 n_sca3 n_sca4 sr_i_1 dt"  # noqa: E501
headers = header.split(" ")

In [ ]:
# 处理 mesh 数据
dfout = []
scan_folder = create_folders(path_out, "0_PureAbsorption")
# read data files and clean it
for master_file_name in tqdm(path_master_file):
    name_folder = master_file_name.parts[-1][:-4]

    dfin = pd.read_csv(master_file_name, index_col=None, header=None, names=headers, sep=r"\s+", comment="#")

    NaNindex = dfin.isnull().any(axis=1).to_numpy().nonzero()[0].tolist()
    dfin.drop(NaNindex, axis=0, inplace=True)

    # 计算 Absorption
    i0 = dfin.loc[:, ["a_i0_1", "a_i0_2"]].sum(axis=1).values
    i1 = dfin.loc[:, ["a_i1_1", "a_i1_2"]].sum(axis=1).values
    dfin["absorption"] = np.log(i0 / i1)  # type: ignore # natural base
    df2out = dfin[columns := ["Pt_No", "tripod_x", "tripod_z", "absorption"]].copy(deep=True)

    # 保存每个能量值下的数据
    df2out.to_csv(Path.joinpath(scan_folder, f"{name_folder}_all.csv"), sep=",", index=False)
    dfout.append(df2out)

    # 新建对应的文件夹以及获取对应的位置
    # 获取切片的引索序列
    scancuttoff_index_list = df2out[(df2out.loc[:, "Pt_No"] == df2out.loc[:, "Pt_No"].min())].index.tolist()
    # scancuttoff_index_list=dfin2[(dfin2.loc[:,'Pt_No']==dfin2.loc[:,'Pt_No'].max())].index.tolist()  # noqa: ERA001
    # print(len(scancuttoff_index_list))  # noqa: ERA001

    # finding global min and max intensity cheapo way for transmission
    vmin = df2out.loc[:, "absorption"].min()
    vmax = df2out.loc[:, "absorption"].max()
    # print(vmin, vmax)  # noqa: ERA001

    for i in trange(len(scancuttoff_index_list) - 1):
        scan_name = f"{name_folder}_{i + 1:04d}"
        path_file_folder = create_folders(scan_folder, name_folder)

        a = scancuttoff_index_list[i]
        b = scancuttoff_index_list[i + 1]

        # 输出每个能量下的每次扫描数据，并画图
        df2out.iloc[a:b, :].to_csv(
            Path.joinpath(path_file_folder, f"{scan_name}.csv"), sep=",", index=True, header=True
        )
        slice_energy = df2out.iloc[a:b, :].pivot(index="tripod_z", columns="tripod_x", values="absorption")
        slice_energy.to_csv(Path.joinpath(path_file_folder, f"{scan_name}_pivot.csv"), sep=",", index=True, header=True)
        figure = data2map(slice_energy, scan_name, todisplay=False, vmin=vmin, vmax=vmax)
        figure.savefig(
            Path.joinpath(path_file_folder, f"{scan_name}.tif"),
            transparent=False,
            pad_inches=0.05,
            bbox_inches="tight",
            dpi=300,
            pil_kwargs={"compression": "tiff_lzw"},
        )
        plt.close()

#### 数据的清洗

In [ ]:
# 裁剪所有 absorption 列到最短长度
min_len = min(len(df) for df in dfout)
Mesh_data = np.array([df.iloc[:min_len, -1].values for df in dfout]).T

# 裁剪前3列位置信息并合并
Mesh_data = pd.concat([dfout[0].iloc[:min_len, :3].reset_index(drop=True), pd.DataFrame(Mesh_data)], axis=1).copy()
Mesh_data.to_csv(
    Path.joinpath(path_out, "Mesh_Absorption_data.csv"),
    sep=",",
    index=False,
    header=[
        r"Pt_No",
        r"tripod_x",
        r"tripod_z",
        r"absorption1",
        r"absorption2",
        r"absorption3",
        r"absorption4",
        r"absorption5",
    ],  # noqa: E501
)

#### PreEdge 和 PostEdge 的拟合

In [ ]:
from scipy.optimize import minimize

# 数据
E_Mn = np.array([6533.0, 6553.2, 6557.5, 6561.5, 6610.8])
Ra_Mn = np.array([0.01, 1.95670, 1.19821, 1.03919, 1.04312])
Rb_Mn = np.array([0.01, 0.531024, 1.04128, 1.42149, 0.983545])
constants = [Ra_Mn, Rb_Mn]


# 拟合函数
def model(params, energy, constants):
    eff1, eff2, deltamu, pre_slope, pre_intercept = params
    Ra, Rb = constants
    return deltamu * (Ra * eff1 + Rb * eff2) + pre_slope * energy + pre_intercept


# 目标函数
def objective(params, energy, constants, data):
    pred = model(params, energy, constants)
    mse = np.mean((data - pred) ** 2)
    eff1, eff2 = params[0], params[1]
    penalty = 100 * (eff1 + eff2 - 1) ** 2
    return mse + penalty


# 初始化参数
initial_guess = [0.5, 0.5, 1.0, 0.0, 0.0]
bounds = [(0, 1), (0, 1), (0, np.inf), (-np.inf, np.inf), (-np.inf, np.inf)]

# 初始化结果容器
params_result = pd.DataFrame(index=Mesh_data.index, columns=["eff1", "eff2", "deltamu", "pre_slope", "pre_intercept"])
absroption_result = pd.DataFrame(index=Mesh_data.index, columns=[f"Normal_{i}" for i in range(Mesh_data.shape[1] - 3)])

# 拟合过程
for i in tqdm(Mesh_data.index):
    temp = Mesh_data.iloc[i, 3:].values
    result = minimize(
        objective, initial_guess, args=(E_Mn, constants, temp), bounds=bounds, method="SLSQP", options={"maxiter": 1000}
    )

    if result.success:
        eff1, eff2, deltamu, pre_slope, pre_intercept = result.x
        params_result.loc[i] = result.x
        fitted_absorption = deltamu * (Ra_Mn * eff1 + Rb_Mn * eff2)
        absroption_result.loc[i] = fitted_absorption[:5]

#### 验证结果的准确度

In [ ]:
# 保存数据
pd.concat([Mesh_data, absroption_result], axis=1, ignore_index=True).to_csv(
    Path.joinpath(path_out, "Mesh_Absorption_results.csv"),
    sep=",",
    index=False,
    header=[
        r"Pt_No",
        r"tripod_x",
        r"tripod_z",
        r"absorption1",
        r"absorption2",
        r"absorption3",
        r"absorption4",
        r"absorption5",
        r"Normal1",
        r"Normal2",
        r"Normal3",
        r"Normal4",
        r"Normal5",
    ],
)
pd.concat([Mesh_data.iloc[:, :3], params_result], axis=1, ignore_index=True).to_csv(
    Path.joinpath(path_out, "Mesh_Absorption_params.csv"),
    sep=",",
    index=False,
    header=[r"Pt_No", r"tripod_x", r"tripod_z", r"eff1", r"eff2", r"deltamu", r"pre_slope", r"pre_intercept"],
)

#### 保存成 tiff 图片

In [ ]:
import tifffile

absroption_result_copy = pd.concat([Mesh_data.iloc[:, :3], absroption_result], axis=1, ignore_index=True)
absroption_result_copy = absroption_result_copy.apply(pd.to_numeric, errors="coerce")


# 每像素 0.5 μm -> 20,000 pixels per cm
resolution = (20000 * 0.32, 10000 * 0.67)  # (x_res, y_res)

scan_folder = create_folders(path_out, "1_Results")
folder_names = [f.parts[-1][:-4] for f in path_master_file]

for idx in trange(3, absroption_result_copy.shape[1]):
    current_folder_name = folder_names[idx - 3]
    path_file_folder = create_folders(scan_folder, current_folder_name)

    columns = absroption_result_copy.columns
    temp = absroption_result_copy.loc[:, [columns[0], columns[1], columns[2], columns[idx]]]
    temp.columns = ["Pt_No", "tripod_x", "tripod_z", "absorption"]

    scancutoff_indices = temp[temp["Pt_No"] == temp["Pt_No"].min()].index.tolist()

    vmin = temp["absorption"].min(skipna=True)
    vmax = temp["absorption"].max(skipna=True)
    fig_list = []
    for i in trange(len(scancutoff_indices) - 1):
        scan_name = f"{current_folder_name}_{i + 1:04d}"
        a, b = scancutoff_indices[i], scancutoff_indices[i + 1]

        slice_df = temp.iloc[a:b, :]
        slice_df.to_csv(Path.joinpath(path_file_folder, f"{scan_name}.csv"), sep=",", index=True, header=True)

        pivot = slice_df.pivot(index="tripod_z", columns="tripod_x", values="absorption")
        pivot.to_csv(Path.joinpath(path_file_folder, f"{scan_name}_pivot.csv"), sep=",", index=True, header=True)
        fig_list.append(pivot.values)
        fig = data2map(pivot, scan_name, todisplay=False, vmin=vmin, vmax=vmax)
        fig.savefig(
            Path.joinpath(path_file_folder, f"{scan_name}.tif"),
            transparent=False,
            pad_inches=0.05,
            bbox_inches="tight",
            dpi=300,
            pil_kwargs={"compression": "tiff_lzw"},
        )
        plt.close()

        tifffile.imwrite(
            Path.joinpath(scan_folder, f"{path_file_folder}.tiff"),
            np.stack(fig_list, axis=0),
            photometric="minisblack",
            compression="lzw",
            resolution=resolution,
            resolutionunit="CENTIMETER",  # or "INCH"
        )

### Mn2

#### 读取数据

In [ ]:
# to read all master files
path_master_file = list(
    Path(
        r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\PaperDos\XAS\Operando\αMnO2\MeshMapping\2023-CLAESS\Data\case1_1stCharge_1stDischarge\UpCell_MnFree\Mn"  # noqa: E501, RUF001
    ).glob("*.dat")
)
header = "Pt_No tripod_z tripod_x energyc a_i0i1_timer a_i0_1 a_i0_2 a_i1_1 a_i1_2 n_timer n_i0_1 n_i0_2 n_i1_1 n_i1_2 n_i2_1 n_i2_2 n_icr n_tcr n_sca1 n_sca2 n_sca3 n_sca4 sr_i_1 dt"  # noqa: E501
headers = header.split(" ")

In [ ]:
# 处理 mesh 数据
dfout = []
scan_folder = create_folders(path_out, "0_PureAbsorption")
# read data files and clean it
for master_file_name in tqdm(path_master_file):
    name_folder = master_file_name.parts[-1][:-4]

    dfin = pd.read_csv(master_file_name, index_col=None, header=None, names=headers, sep=r"\s+", comment="#")

    NaNindex = dfin.isnull().any(axis=1).to_numpy().nonzero()[0].tolist()
    dfin.drop(NaNindex, axis=0, inplace=True)

    # 计算 Absorption
    i0 = dfin.loc[:, ["a_i0_1", "a_i0_2"]].sum(axis=1).values
    i1 = dfin.loc[:, ["a_i1_1", "a_i1_2"]].sum(axis=1).values
    dfin["absorption"] = np.log(i0 / i1)  # type: ignore # natural base
    df2out = dfin[columns := ["Pt_No", "tripod_x", "tripod_z", "absorption"]].copy(deep=True)

    # 保存每个能量值下的数据
    df2out.to_csv(Path.joinpath(scan_folder, f"{name_folder}_all.csv"), sep=",", index=False)
    dfout.append(df2out)

    # 新建对应的文件夹以及获取对应的位置
    # 获取切片的引索序列
    scancuttoff_index_list = df2out[(df2out.loc[:, "Pt_No"] == df2out.loc[:, "Pt_No"].min())].index.tolist()
    # scancuttoff_index_list=dfin2[(dfin2.loc[:,'Pt_No']==dfin2.loc[:,'Pt_No'].max())].index.tolist()  # noqa: ERA001
    # print(len(scancuttoff_index_list))  # noqa: ERA001

    # finding global min and max intensity cheapo way for transmission
    vmin = df2out.loc[:, "absorption"].min()
    vmax = df2out.loc[:, "absorption"].max()
    # print(vmin, vmax)  # noqa: ERA001

    for i in trange(len(scancuttoff_index_list) - 1):
        scan_name = f"{name_folder}_{i + 1:04d}"
        path_file_folder = create_folders(scan_folder, name_folder)

        a = scancuttoff_index_list[i]
        b = scancuttoff_index_list[i + 1]

        # 输出每个能量下的每次扫描数据，并画图
        df2out.iloc[a:b, :].to_csv(
            Path.joinpath(path_file_folder, f"{scan_name}.csv"), sep=",", index=True, header=True
        )
        slice_energy = df2out.iloc[a:b, :].pivot(index="tripod_z", columns="tripod_x", values="absorption")
        slice_energy.to_csv(Path.joinpath(path_file_folder, f"{scan_name}_pivot.csv"), sep=",", index=True, header=True)
        figure = data2map(slice_energy, scan_name, todisplay=False, vmin=vmin, vmax=vmax)
        figure.savefig(
            Path.joinpath(path_file_folder, f"{scan_name}.tif"),
            transparent=False,
            pad_inches=0.05,
            bbox_inches="tight",
            dpi=300,
            pil_kwargs={"compression": "tiff_lzw"},
        )
        plt.close()

#### 数据的清洗

In [ ]:
# 裁剪所有 absorption 列到最短长度
min_len = min(len(df) for df in dfout)
Mesh_data = np.array([df.iloc[:min_len, -1].values for df in dfout]).T

# 裁剪前3列位置信息并合并
Mesh_data = pd.concat([dfout[0].iloc[:min_len, :3].reset_index(drop=True), pd.DataFrame(Mesh_data)], axis=1).copy()
Mesh_data.to_csv(
    Path.joinpath(path_out, "Mesh_Absorption_data.csv"),
    sep=",",
    index=False,
    header=[
        r"Pt_No",
        r"tripod_x",
        r"tripod_z",
        r"absorption1",
        r"absorption2",
        r"absorption3",
        r"absorption4",
        r"absorption5",
    ],  # noqa: E501
)

#### PreEdge 和 PostEdge 的拟合

In [ ]:
from scipy.optimize import minimize

# 数据
E_Mn = np.array([6533.0, 6553.2, 6557.5, 6561.5, 6610.8])
Ra_Mn = np.array([0.01, 1.95670, 1.19821, 1.03919, 1.04312])
Rb_Mn = np.array([0.01, 0.531024, 1.04128, 1.42149, 0.983545])
constants = [Ra_Mn, Rb_Mn]


# 拟合函数
def model(params, energy, constants):
    eff1, eff2, deltamu, pre_slope, pre_intercept = params
    Ra, Rb = constants
    return deltamu * (Ra * eff1 + Rb * eff2) + pre_slope * energy + pre_intercept


# 目标函数
def objective(params, energy, constants, data):
    pred = model(params, energy, constants)
    mse = np.mean((data - pred) ** 2)
    eff1, eff2 = params[0], params[1]
    penalty = 100 * (eff1 + eff2 - 1) ** 2
    return mse + penalty


# 初始化参数
initial_guess = [0.5, 0.5, 1.0, 0.0, 0.0]
bounds = [(0, 1), (0, 1), (0, np.inf), (-np.inf, np.inf), (-np.inf, np.inf)]

# 初始化结果容器
params_result = pd.DataFrame(index=Mesh_data.index, columns=["eff1", "eff2", "deltamu", "pre_slope", "pre_intercept"])
absroption_result = pd.DataFrame(index=Mesh_data.index, columns=[f"Normal_{i}" for i in range(Mesh_data.shape[1] - 3)])

# 拟合过程
for i in tqdm(Mesh_data.index):
    temp = Mesh_data.iloc[i, 3:].values
    result = minimize(
        objective, initial_guess, args=(E_Mn, constants, temp), bounds=bounds, method="SLSQP", options={"maxiter": 1000}
    )

    if result.success:
        eff1, eff2, deltamu, pre_slope, pre_intercept = result.x
        params_result.loc[i] = result.x
        fitted_absorption = deltamu * (Ra_Mn * eff1 + Rb_Mn * eff2)
        absroption_result.loc[i] = fitted_absorption[:5]

#### 验证结果的准确度

In [ ]:
# 保存数据
pd.concat([Mesh_data, absroption_result], axis=1, ignore_index=True).to_csv(
    Path.joinpath(path_out, "Mesh_Absorption_results.csv"),
    sep=",",
    index=False,
    header=[
        r"Pt_No",
        r"tripod_x",
        r"tripod_z",
        r"absorption1",
        r"absorption2",
        r"absorption3",
        r"absorption4",
        r"absorption5",
        r"Normal1",
        r"Normal2",
        r"Normal3",
        r"Normal4",
        r"Normal5",
    ],
)
pd.concat([Mesh_data.iloc[:, :3], params_result], axis=1, ignore_index=True).to_csv(
    Path.joinpath(path_out, "Mesh_Absorption_params.csv"),
    sep=",",
    index=False,
    header=[r"Pt_No", r"tripod_x", r"tripod_z", r"eff1", r"eff2", r"deltamu", r"pre_slope", r"pre_intercept"],
)

#### 保存成 tiff 图片

In [ ]:
import tifffile

absroption_result_copy = pd.concat([Mesh_data.iloc[:, :3], absroption_result], axis=1, ignore_index=True)
absroption_result_copy = absroption_result_copy.apply(pd.to_numeric, errors="coerce")


# 每像素 0.5 μm -> 20,000 pixels per cm
resolution = (20000 * 0.32, 10000 * 0.67)  # (x_res, y_res)

scan_folder = create_folders(path_out, "1_Results")
folder_names = [f.parts[-1][:-4] for f in path_master_file]

for idx in trange(3, absroption_result_copy.shape[1]):
    current_folder_name = folder_names[idx - 3]
    path_file_folder = create_folders(scan_folder, current_folder_name)

    columns = absroption_result_copy.columns
    temp = absroption_result_copy.loc[:, [columns[0], columns[1], columns[2], columns[idx]]]
    temp.columns = ["Pt_No", "tripod_x", "tripod_z", "absorption"]

    scancutoff_indices = temp[temp["Pt_No"] == temp["Pt_No"].min()].index.tolist()

    vmin = temp["absorption"].min(skipna=True)
    vmax = temp["absorption"].max(skipna=True)
    fig_list = []
    for i in trange(len(scancutoff_indices) - 1):
        scan_name = f"{current_folder_name}_{i + 1:04d}"
        a, b = scancutoff_indices[i], scancutoff_indices[i + 1]

        slice_df = temp.iloc[a:b, :]
        slice_df.to_csv(Path.joinpath(path_file_folder, f"{scan_name}.csv"), sep=",", index=True, header=True)

        pivot = slice_df.pivot(index="tripod_z", columns="tripod_x", values="absorption")
        pivot.to_csv(Path.joinpath(path_file_folder, f"{scan_name}_pivot.csv"), sep=",", index=True, header=True)
        fig_list.append(pivot.values)
        fig = data2map(pivot, scan_name, todisplay=False, vmin=vmin, vmax=vmax)
        fig.savefig(
            Path.joinpath(path_file_folder, f"{scan_name}.tif"),
            transparent=False,
            pad_inches=0.05,
            bbox_inches="tight",
            dpi=300,
            pil_kwargs={"compression": "tiff_lzw"},
        )
        plt.close()

        tifffile.imwrite(
            Path.joinpath(scan_folder, f"{path_file_folder}.tiff"),
            np.stack(fig_list, axis=0),
            photometric="minisblack",
            compression="lzw",
            resolution=resolution,
            resolutionunit="CENTIMETER",  # or "INCH"
        )

### Mn1

#### 读取数据

In [ ]:
# to read all master files
path_master_file = list(
    Path(
        r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\PaperDos\XAS\Operando\αMnO2\MeshMapping\2023-CLAESS\Data\case1_1stCharge_1stDischarge\DownCell_Mn\Mn"  # noqa: E501, RUF001
    ).glob("*.dat")
)
header = "Pt_No tripod_z tripod_x energyc a_i0i1_timer a_i0_1 a_i0_2 a_i1_1 a_i1_2 n_timer n_i0_1 n_i0_2 n_i1_1 n_i1_2 n_i2_1 n_i2_2 n_icr n_tcr n_sca1 n_sca2 n_sca3 n_sca4 sr_i_1 dt"  # noqa: E501
headers = header.split(" ")

In [ ]:
# 处理 mesh 数据
dfout = []
scan_folder = create_folders(path_out, "0_PureAbsorption")
# read data files and clean it
for master_file_name in tqdm(path_master_file):
    name_folder = master_file_name.parts[-1][:-4]

    dfin = pd.read_csv(master_file_name, index_col=None, header=None, names=headers, sep=r"\s+", comment="#")

    NaNindex = dfin.isnull().any(axis=1).to_numpy().nonzero()[0].tolist()
    dfin.drop(NaNindex, axis=0, inplace=True)

    # 计算 Absorption
    i0 = dfin.loc[:, ["a_i0_1", "a_i0_2"]].sum(axis=1).values
    i1 = dfin.loc[:, ["a_i1_1", "a_i1_2"]].sum(axis=1).values
    dfin["absorption"] = np.log(i0 / i1)  # type: ignore # natural base
    df2out = dfin[columns := ["Pt_No", "tripod_x", "tripod_z", "absorption"]].copy(deep=True)

    # 保存每个能量值下的数据
    df2out.to_csv(Path.joinpath(scan_folder, f"{name_folder}_all.csv"), sep=",", index=False)
    dfout.append(df2out)

    # 新建对应的文件夹以及获取对应的位置
    # 获取切片的引索序列
    scancuttoff_index_list = df2out[(df2out.loc[:, "Pt_No"] == df2out.loc[:, "Pt_No"].min())].index.tolist()
    # scancuttoff_index_list=dfin2[(dfin2.loc[:,'Pt_No']==dfin2.loc[:,'Pt_No'].max())].index.tolist()  # noqa: ERA001
    # print(len(scancuttoff_index_list))  # noqa: ERA001

    # finding global min and max intensity cheapo way for transmission
    vmin = df2out.loc[:, "absorption"].min()
    vmax = df2out.loc[:, "absorption"].max()
    # print(vmin, vmax)  # noqa: ERA001

    for i in trange(len(scancuttoff_index_list) - 1):
        scan_name = f"{name_folder}_{i + 1:04d}"
        path_file_folder = create_folders(scan_folder, name_folder)

        a = scancuttoff_index_list[i]
        b = scancuttoff_index_list[i + 1]

        # 输出每个能量下的每次扫描数据，并画图
        df2out.iloc[a:b, :].to_csv(
            Path.joinpath(path_file_folder, f"{scan_name}.csv"), sep=",", index=True, header=True
        )
        slice_energy = df2out.iloc[a:b, :].pivot(index="tripod_z", columns="tripod_x", values="absorption")
        slice_energy.to_csv(Path.joinpath(path_file_folder, f"{scan_name}_pivot.csv"), sep=",", index=True, header=True)
        figure = data2map(slice_energy, scan_name, todisplay=False, vmin=vmin, vmax=vmax)
        figure.savefig(
            Path.joinpath(path_file_folder, f"{scan_name}.tif"),
            transparent=False,
            pad_inches=0.05,
            bbox_inches="tight",
            dpi=300,
            pil_kwargs={"compression": "tiff_lzw"},
        )
        plt.close()

#### 数据的清洗

In [ ]:
Mesh_data = np.asarray(list(dfout))[:, :, -1]
Mesh_data = pd.concat([dfout[0].iloc[:, :3], pd.DataFrame(Mesh_data.T)], axis=1, ignore_index=True)
Mesh_data.to_csv(
    Path.joinpath(path_out, "Mesh_Absorption_data.csv"),
    sep=",",
    index=False,
    header=[
        r"Pt_No",
        r"tripod_x",
        r"tripod_z",
        r"absorption1",
        r"absorption2",
        r"absorption3",
        r"absorption4",
        r"absorption5",
    ],  # noqa: E501
)

#### PreEdge 和 PostEdge 的拟合

In [ ]:
from scipy.optimize import minimize

# 数据
E_Mn = np.array([6533.0, 6553.2, 6557.5, 6561.5, 6610.8])
Ra_Mn = np.array([0.01, 1.95670, 1.19821, 1.03919, 1.04312])
Rb_Mn = np.array([0.01, 0.531024, 1.04128, 1.42149, 0.983545])
constants = [Ra_Mn, Rb_Mn]


# 拟合函数
def model(params, energy, constants):
    eff1, eff2, deltamu, pre_slope, pre_intercept = params
    Ra, Rb = constants
    return deltamu * (Ra * eff1 + Rb * eff2) + pre_slope * energy + pre_intercept


# 目标函数
def objective(params, energy, constants, data):
    pred = model(params, energy, constants)
    mse = np.mean((data - pred) ** 2)
    eff1, eff2 = params[0], params[1]
    penalty = 100 * (eff1 + eff2 - 1) ** 2
    return mse + penalty


# 初始化参数
initial_guess = [0.5, 0.5, 1.0, 0.0, 0.0]
bounds = [(0, 1), (0, 1), (0, np.inf), (-np.inf, np.inf), (-np.inf, np.inf)]

# 初始化结果容器
params_result = pd.DataFrame(index=Mesh_data.index, columns=["eff1", "eff2", "deltamu", "pre_slope", "pre_intercept"])
absroption_result = pd.DataFrame(index=Mesh_data.index, columns=[f"Normal_{i}" for i in range(Mesh_data.shape[1] - 3)])

# 拟合过程
for i in tqdm(Mesh_data.index):
    temp = Mesh_data.iloc[i, 3:].values
    result = minimize(
        objective, initial_guess, args=(E_Mn, constants, temp), bounds=bounds, method="SLSQP", options={"maxiter": 1000}
    )

    if result.success:
        eff1, eff2, deltamu, pre_slope, pre_intercept = result.x
        params_result.loc[i] = result.x
        fitted_absorption = deltamu * (Ra_Mn * eff1 + Rb_Mn * eff2)
        absroption_result.loc[i] = fitted_absorption[:5]

#### 验证结果的准确度

In [ ]:
# 保存数据
pd.concat([Mesh_data, absroption_result], axis=1, ignore_index=True).to_csv(
    Path.joinpath(path_out, "Mesh_Absorption_results.csv"),
    sep=",",
    index=False,
    header=[
        r"Pt_No",
        r"tripod_x",
        r"tripod_z",
        r"absorption1",
        r"absorption2",
        r"absorption3",
        r"absorption4",
        r"absorption5",
        r"Normal1",
        r"Normal2",
        r"Normal3",
        r"Normal4",
        r"Normal5",
    ],
)
pd.concat([Mesh_data.iloc[:, :3], params_result], axis=1, ignore_index=True).to_csv(
    Path.joinpath(path_out, "Mesh_Absorption_params.csv"),
    sep=",",
    index=False,
    header=[r"Pt_No", r"tripod_x", r"tripod_z", r"eff1", r"eff2", r"deltamu", r"pre_slope", r"pre_intercept"],
)

#### 保存成 tiff 图片

In [ ]:
import tifffile

absroption_result_copy = pd.concat([Mesh_data.iloc[:, :3], absroption_result], axis=1, ignore_index=True)
absroption_result_copy = absroption_result_copy.apply(pd.to_numeric, errors="coerce")


# 每像素 0.5 μm -> 20,000 pixels per cm
resolution = (20000 * 0.32, 10000 * 0.67)  # (x_res, y_res)

scan_folder = create_folders(path_out, "1_Results")
folder_names = [f.parts[-1][:-4] for f in path_master_file]

for idx in trange(3, absroption_result_copy.shape[1]):
    current_folder_name = folder_names[idx - 3]
    path_file_folder = create_folders(scan_folder, current_folder_name)

    columns = absroption_result_copy.columns
    temp = absroption_result_copy.loc[:, [columns[0], columns[1], columns[2], columns[idx]]]
    temp.columns = ["Pt_No", "tripod_x", "tripod_z", "absorption"]

    scancutoff_indices = temp[temp["Pt_No"] == temp["Pt_No"].min()].index.tolist()

    vmin = temp["absorption"].min(skipna=True)
    vmax = temp["absorption"].max(skipna=True)
    fig_list = []
    for i in trange(len(scancutoff_indices) - 1):
        scan_name = f"{current_folder_name}_{i + 1:04d}"
        a, b = scancutoff_indices[i], scancutoff_indices[i + 1]

        slice_df = temp.iloc[a:b, :]
        slice_df.to_csv(Path.joinpath(path_file_folder, f"{scan_name}.csv"), sep=",", index=True, header=True)

        pivot = slice_df.pivot(index="tripod_z", columns="tripod_x", values="absorption")
        pivot.to_csv(Path.joinpath(path_file_folder, f"{scan_name}_pivot.csv"), sep=",", index=True, header=True)
        fig_list.append(pivot.values)
        fig = data2map(pivot, scan_name, todisplay=False, vmin=vmin, vmax=vmax)
        fig.savefig(
            Path.joinpath(path_file_folder, f"{scan_name}.tif"),
            transparent=False,
            pad_inches=0.05,
            bbox_inches="tight",
            dpi=300,
            pil_kwargs={"compression": "tiff_lzw"},
        )
        plt.close()

        tifffile.imwrite(
            Path.joinpath(scan_folder, f"{path_file_folder}.tiff"),
            np.stack(fig_list, axis=0),
            photometric="minisblack",
            compression="lzw",
            resolution=resolution,
            resolutionunit="CENTIMETER",  # or "INCH"
        )